In [ ]:
folder_path_untreated = r"C:\\Users\\danwo\Desktop\\ZS0001_TI_25XW_Rhodopsin_Au"
folder_path_treated = r"C:\\Users\\danwo\Desktop\\ZS0003_TI_25XW_Sample1_GNP_MB"



def get_frame_from_folder(folder_path, frame_index):
    """
    Devuelve la imagen correspondiente al fotograma (frame_index) de una carpeta de imágenes ordenadas.

    Parámetros:
    - folder_path: Ruta a la carpeta con las imágenes.
    - frame_index: Índice del fotograma deseado (empezando en 0).

    Retorna:
    - img: Imagen en escala de grises como array de numpy.
    """
    import os
    import cv2

    # Listar y ordenar los archivos de imagen
    image_files = sorted([f for f in os.listdir(folder_path) if f.endswith(('.png', '.jpg', '.tif'))])
    if not image_files:
        raise ValueError("No se encontraron imágenes en la carpeta")
    if frame_index < 0 or frame_index >= len(image_files):
        raise IndexError("El índice del fotograma está fuera de rango")
    
    image_path = os.path.join(folder_path, image_files[frame_index])
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        raise ValueError(f"No se pudo cargar la imagen: {image_path}")
    return img



import os
import matplotlib.pyplot as plt

def save_result_figure(fig, image_type, filename):
    """
    Guarda la figura en results/treated o results/untreated según image_type.
    - fig: objeto matplotlib.figure.Figure
    - image_type: 'treated' o 'untreated'
    - filename: nombre del archivo (ej: 'orientacion.png')
    """
    base_dir = os.path.join(os.getcwd(), "results", image_type)
    os.makedirs(base_dir, exist_ok=True)
    save_path = os.path.join(base_dir, filename)
    fig.savefig(save_path, bbox_inches='tight')
    print(f"Figura guardada en: {save_path}")
    plt.close(fig)





In [ ]:
image= get_frame_from_folder(folder_path_untreated, 0) 



import matplotlib
import matplotlib.colors
import matplotlib.pyplot as plt
import numpy as np
import orientationpy
import tifffile
import matplotlib.image as mpimg
import cv2

if image is None:
    raise FileNotFoundError(f"No se pudo cargar la imagen")

print(f"Dimensiones de la imagen: {image.shape}")

# Mostrar la imagen original
plt.figure(figsize=(10, 4))
plt.imshow(image, cmap="gray")
plt.title("Imagen original")
plt.show()


# Calcular gradientes con diferentes métodos
methods = ["finite_difference", "gaussian", "splines"]
fig, axes = plt.subplots(len(methods), 2, figsize=(10, 10))

for i, mode in enumerate(methods):
    gradients = orientationpy.computeGradient(image, mode=mode)
    Gy, Gx = gradients[-2:]  # Extraer solo Gy y Gx
    
    # Mostrar gradientes
    axes[i, 0].imshow(Gy, cmap="coolwarm", vmin=-64, vmax=64)
    axes[i, 0].set_title(f"{mode} - Gy")
    
    axes[i, 1].imshow(Gx, cmap="coolwarm", vmin=-64, vmax=64)
    axes[i, 1].set_title(f"{mode} - Gx")

plt.tight_layout()
plt.show()

# Calcular el tensor de estructura
structureTensor = orientationpy.computeStructureTensor([Gy, Gx], sigma=2)

# Calcular la dirección y orientación
orientations = orientationpy.computeOrientation(structureTensor)
directionality = np.linalg.norm(structureTensor, axis=0)

# Asegurar valores positivos para normalización logarítmica
directionality = np.where(~np.isfinite(directionality) | (directionality <= 0), 1e-10, directionality)

plt.figure(figsize=(10, 4))

# Mostrar intensidad normalizada

plt.imshow(directionality / directionality.max(), vmin=0, vmax=1)
plt.colorbar(shrink=0.7)
plt.title("Intensity Normalised")
plt.show()
# Mostrar dirección con normalización logarítmica
plt.figure(figsize=(10, 4))
plt.imshow(directionality, norm=matplotlib.colors.LogNorm(vmin=1e-10, vmax=directionality.max()))
plt.title("Directionality Normalised")
plt.colorbar(shrink=0.7)

plt.tight_layout()
plt.show()

# Normalizar dirección
vmin, vmax = 10, np.max(directionality)
directionality = np.clip(directionality, vmin, vmax)
directionality = np.log(directionality)
directionality -= directionality.min()
directionality /= directionality.max()
directionality[image == 0] = 0

# Visualizar imagen con orientación HSV superpuesta
plt.figure(figsize=(6, 6))
plt.title("Overlay with orientation \n Greyscale image with HSV orientations overlaid")

plt.imshow(image, cmap="Greys_r", vmin=0)
plt.imshow(
    orientations["theta"],
    cmap="hsv",
    alpha=directionality * 0.5,
    vmin=-90,
    vmax=90,
)
plt.colorbar(shrink=0.7)
plt.show()

# Visualizar el tensor de estructura
plt.imshow(structureTensor.sum(axis=0), cmap="viridis")
plt.title("Tensor de estructura")
plt.colorbar()
plt.show()

# Convertir imagen a HSV
imDisplayHSV = np.zeros((image.shape[0], image.shape[1], 3), dtype="f4")
imDisplayHSV[:, :, 0] = (orientations["theta"] + 90) / 180  # Hue
imDisplayHSV[:, :, 1] = directionality  # Saturation
imDisplayHSV[:, :, 2] = image / image.max()  # Value

fig, ax = plt.subplots()
fig.suptitle("Image-orientation composition")
ax.set_title("Hue = Orientation\nSaturation = log(Directionality)\nV = image greylevels")
ax.imshow(matplotlib.colors.hsv_to_rgb(imDisplayHSV))

cmap = matplotlib.cm.hsv
norm = matplotlib.colors.Normalize(vmin=-90, vmax=90)
fig.colorbar(
    matplotlib.cm.ScalarMappable(norm=norm, cmap=cmap),
    ax=ax,
    orientation="vertical",
    label="Degrees from horizontal",
    shrink=0.7,
)

plt.show()

# Cálculo de orientación en bloques
boxSizePixels = 7
structureTensorBoxes = orientationpy.computeStructureTensorBoxes(
    [Gy, Gx], [boxSizePixels, boxSizePixels]
)
intensityBoxes = orientationpy.computeIntensity(structureTensorBoxes)
orientationsBoxes = orientationpy.computeOrientation(structureTensorBoxes, mode="fiber")
intensityBoxes /= intensityBoxes.max()

# Calcular centros de bloques
boxCentresY = np.arange(orientationsBoxes["theta"].shape[0]) * boxSizePixels + boxSizePixels // 2
boxCentresX = np.arange(orientationsBoxes["theta"].shape[1]) * boxSizePixels + boxSizePixels // 2

# Calcular componentes del vector
boxVectorsYX = orientationpy.anglesToVectors(orientationsBoxes)
boxVectorsYX[:, intensityBoxes < 0.05] = 0.0

plt.title("Local orientation vector in boxes")
plt.imshow(image, cmap="Greys_r", vmin=0)

plt.quiver(
    boxCentresX,
    boxCentresY,
    boxVectorsYX[1],
    boxVectorsYX[0],
    angles="xy",
    scale_units="xy",
    color="r",
    headwidth=0,
    headlength=0,
    headaxislength=1,
)
plt.show()


In [ ]:
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
from scipy.fftpack import fftn, fftshift, ifftn
from scipy.ndimage import rotate
from scipy.ndimage import binary_opening

# Filtrar puntos dispersos en la máscara
def clean_mask(mask):
    cleaned_mask = binary_opening(mask, structure=np.ones((3, 3)))  # Filtro morfológico
    return cleaned_mask

# Función para cargar una imagen en escala de grises
def load_image(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        raise ValueError(f"No se pudo cargar la imagen: {image_path}")
    return img

# Función para calcular la Transformada de Fourier 2D
def compute_fft_2d(img):
    f_transform = fftn(img)  
    f_transform_shifted = fftshift(f_transform)  
    magnitude_spectrum = np.log1p(np.abs(f_transform_shifted))  
    return f_transform_shifted, magnitude_spectrum

# Filtro por porcentaje de energía
def filter_by_energy(f_transform, energy_percent=20):
    magnitude = np.abs(f_transform)
    total_energy = np.sum(magnitude)  # Energía total de la imagen en la frecuencia
    sorted_magnitude = np.sort(magnitude.ravel())[::-1]  # Ordenamos de mayor a menor
    cumulative_energy = np.cumsum(sorted_magnitude)  # Suma acumulativa de energía
    
    # Encontramos el umbral donde alcanzamos el porcentaje de energía deseado
    threshold_index = np.searchsorted(cumulative_energy, total_energy * (energy_percent / 100))
    threshold_value = sorted_magnitude[threshold_index]

    # Creamos la máscara conservando solo los valores con suficiente energía
    mask = magnitude >= threshold_value
    filtered_transform = f_transform * mask  # Aplicamos el filtro
    
    print(f"Umbral de energía aplicado: {threshold_value:.4f} (Conserva {energy_percent}% de la energía)")
    return filtered_transform, mask

# Calcular la dirección principal del espectro filtrado
def calculate_main_directions(mask, magnitude=None, num_directions=3):
    """
    Calcula las direcciones principales del espectro filtrado.
    
    Parámetros:
        mask (ndarray): Máscara binaria del espectro.
        magnitude (ndarray, opcional): Magnitud del espectro para ponderar los puntos.
        num_directions (int): Número de direcciones principales a calcular.
    
    Retorna:
        list: Lista de direcciones principales en grados.
    """
    h, w = mask.shape
    cy, cx = h // 2, w // 2  # Centro del espectro

    # Filtrar puntos dispersos
    mask = clean_mask(mask)

    # Coordenadas de los puntos activados en la máscara
    y, x = np.nonzero(mask)
    if len(x) == 0 or len(y) == 0:
        print("La máscara no tiene puntos activados.")
        return [0]  # O algún valor predeterminado

    y = y - cy
    x = x - cx

    # Calcular la relación de aspecto de los puntos activados
    range_x = np.max(x) - np.min(x)
    range_y = np.max(y) - np.min(y)
    aspect_ratio = max(range_x, 1) / max(range_y, 1)  # Evitar división por cero

    # Depuración: Imprimir los valores
    print(f"range_x: {range_x}, range_y: {range_y}, aspect_ratio: {aspect_ratio}")


    # Decidir el método basado en la relación de aspecto
    if aspect_ratio > 1.7 or aspect_ratio < 0.6: # Una sola línea preferente
        print("Usando método: least_squares")
        # Ajustar una línea usando mínimos cuadrados
        A = np.vstack([x, np.ones(len(x))]).T
        m, _ = np.linalg.lstsq(A, y, rcond=None)[0]
        main_direction = np.arctan(m)  # Ángulo en radianes
        return [np.degrees(main_direction)]  # Convertir a grados y devolver en lista
    else:
        print("Usando método: covariance")
        # Calcular la matriz de covarianza
        if magnitude is not None:
            magnitude = np.abs(magnitude)  # Asegurarse de que no haya valores negativos
            weights = magnitude[mask]
            weights /= np.sum(weights)  # Normalizar los pesos
            cov_matrix = np.cov(x, y, aweights=weights)
        else:
            cov_matrix = np.cov(x, y)

        # Calcular valores y vectores propios
        eigenvalues, eigenvectors = np.linalg.eig(cov_matrix)

        # Ordenar los valores propios y vectores propios en orden descendente
        sorted_indices = np.argsort(eigenvalues)[::-1]
        eigenvectors = eigenvectors[:, sorted_indices]

        # Calcular las direcciones principales
        directions = []
        for i in range(min(num_directions, len(eigenvalues))):
            angle = np.arctan2(eigenvectors[1, i], eigenvectors[0, i])  # Ángulo en radianes
            directions.append(np.degrees(angle))  # Convertir a grados

        # Si no hay suficientes direcciones, rellena con direcciones uniformemente distribuidas
        while len(directions) < num_directions:
            directions.append((360 / num_directions) * len(directions))

        return directions

# Filtro de dos líneas delgadas rotadas
def filter_two_lines_rotated(f_transform, angle):
    h, w = f_transform.shape
    cy, cx = h // 2, w // 2  # Centro del espectro

    # Crear una máscara con dos líneas delgadas
    mask = np.zeros((h, w), dtype=np.uint8)

    # Línea horizontal
    mask[cy - 1:cy + 2, :] = 1  # Grosor de 3 píxeles

    # Línea vertical
    mask[:, cx - 1:cx + 2] = 1  # Grosor de 3 píxeles

    # Rotar la máscara
    rotated_mask = rotate(mask, angle, reshape=False, order=1)
    filtered_transform = f_transform * rotated_mask
    return filtered_transform, rotated_mask

# Función principal para procesar todas las imágenes en una carpeta
def process_images_with_rotated_filter(folder_path):
    image_files = sorted([f for f in os.listdir(folder_path) if f.endswith(('.png', '.jpg', '.tif'))])
    if not image_files:
        raise ValueError("No se encontraron imágenes en la carpeta")

    for image_file in image_files:
        image_path = os.path.join(folder_path, image_file)
        img = load_image(image_path)  # Cargar la imagen
        f_transform_shifted, magnitude_spectrum = compute_fft_2d(img)  # Transformada de Fourier 2D
        
        # Aplicar el filtro por energía al 20%
        filtered_transform_energy, mask_energy = filter_by_energy(f_transform_shifted, energy_percent=20)

        # Calcular las direcciones principales del espectro filtrado
        main_directions = calculate_main_directions(mask_energy, magnitude=np.abs(f_transform_shifted), num_directions=3)
        print(f"Direcciones principales calculadas: {main_directions}")

        # Aplicar el filtro de dos líneas rotadas para cada dirección
        combined_filtered_transform = np.zeros_like(f_transform_shifted)
        combined_rotated_mask = np.zeros_like(mask_energy, dtype=np.uint8)  # Asegurar que sea uint8
        for direction in main_directions:
            filtered_transform_lines, rotated_mask = filter_two_lines_rotated(f_transform_shifted, direction)
            combined_filtered_transform += filtered_transform_lines
            combined_rotated_mask += rotated_mask

        # Espectros filtrados para visualizar
        filtered_magnitude_spectrum_energy = np.log1p(np.abs(filtered_transform_energy))
        filtered_magnitude_spectrum_lines = np.log1p(np.abs(combined_filtered_transform))
        
        # Reconstrucción de la imagen desde la transformada filtrada
        reconstructed_img_energy = np.abs(ifftn(filtered_transform_energy))
        reconstructed_img_lines = np.abs(ifftn(combined_filtered_transform))

        # Visualización de resultados
        plt.figure(figsize=(20, 8))
        plt.subplot(2, 4, 1)
        plt.imshow(img, cmap='gray')
        plt.title('Imagen Original')
        plt.axis('off')
        
        plt.subplot(2, 4, 2)
        plt.imshow(magnitude_spectrum, cmap='turbo')
        plt.title('Transformada de Fourier 2D')
        plt.axis('off')
        
        plt.subplot(2, 4, 3)
        plt.imshow(mask_energy, cmap='gray')
        plt.title('Máscara del Filtro (20% Energía)')
        plt.axis('off')
        
        plt.subplot(2, 4, 4)
        plt.imshow(filtered_magnitude_spectrum_energy, cmap='turbo')
        plt.title('Espectro Filtrado (20% Energía)')
        plt.axis('off')
        
        plt.subplot(2, 4, 5)
        plt.imshow(combined_rotated_mask, cmap='gray')
        plt.title(f'Máscara Rotada ({len(main_directions)} direcciones)')
        plt.axis('off')
        
        plt.subplot(2, 4, 6)
        plt.imshow(filtered_magnitude_spectrum_lines, cmap='turbo')
        plt.title('Espectro Filtrado (Líneas Rotadas)')
        plt.axis('off')
        
        plt.subplot(2, 4, 7)
        plt.imshow(reconstructed_img_energy, cmap='gray')
        plt.title('Imagen Reconstruida (20% Energía)')
        plt.axis('off')
        
        plt.subplot(2, 4, 8)
        plt.imshow(reconstructed_img_lines, cmap='gray')
        plt.title('Imagen Reconstruida (Líneas Rotadas)')
        plt.axis('off')
        
        plt.tight_layout()
        plt.show()
# Ejecutar el procesamiento con filtro rotado
process_images_with_rotated_filter(folder_path_untreated)

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.filters import sobel
from skimage.feature import graycomatrix, graycoprops
from scipy.ndimage import gaussian_filter

def load_image(image_path):
    try:
        img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            raise ValueError("Image not found or unable to load.")
        return img
    except Exception as e:
        print(f"Error loading image: {e}")
        return None

def compute_orientation_tensor(image):
    """Calculate the structure tensor and orientation map."""
    sobel_x = sobel(image, axis=0) # Compute the gradient along the x-axis
    sobel_y = sobel(image, axis=1)
    
    J11 = gaussian_filter(sobel_x * sobel_x, sigma=1)
    J22 = gaussian_filter(sobel_y * sobel_y, sigma=1)
    J12 = gaussian_filter(sobel_x * sobel_y, sigma=1)
    
    theta = 0.5 * np.arctan2(2 * J12, (J11 - J22))
    doi_map = np.sqrt(J12**2 + (J11 - J22)**2) / (J11 + J22 + 1e-10)  # Normalized DOI map
    return theta, doi_map

def histogram_of_orientations(theta):
    """Compute histogram of preferred orientations."""
    angles = theta.flatten()
    hist, bins = np.histogram(angles, bins=36, range=(-np.pi/2, np.pi/2), density=True)
    return hist, bins

def compute_doi(hist):
    """Compute the Degree of Isotropy (DoI)."""
    doi = np.std(hist) / np.max(hist) if np.max(hist) > 0 else 0
    return doi

def compute_shg_intensity(image):
    """Compute the mean intensity of SHG signal."""
    return np.mean(image)

def texture_analysis(image):
    """Compute texture features using GLCM (Grey-Level Co-Occurrence Matrix)."""
    glcm = graycomatrix(image, distances=[1], angles=[0], levels=256, symmetric=True, normed=True)
    contrast = graycoprops(glcm, 'contrast')[0, 0]
    entropy = -np.sum(glcm * np.log2(glcm + 1e-10))
    return contrast, entropy

def process_shg_image(image_path, region_label="Region"): 
    """Process an SHG image and extract features."""
    img = load_image(image_path)
    if img is None:
        return None
    
    theta, doi_map = compute_orientation_tensor(img)
    hist, bins = histogram_of_orientations(theta)
    doi = compute_doi(hist)
    shg_intensity = compute_shg_intensity(img)
    contrast, entropy = texture_analysis(img)
    
    results = {
        "DoI": doi,
        "SHG Intensity": shg_intensity,
        "Contrast": contrast,
        "Entropy": entropy,
        "Orientation Histogram": hist,
        "Bins": bins,
        "DoI Map": doi_map
    }
    
    # Plot DoI map
    plt.figure(figsize=(6,6))
    plt.imshow(doi_map, cmap='jet', interpolation='nearest')
    plt.colorbar(label="Degree of Isotropy (DoI)")
    plt.title(f"DoI Map - {region_label}")
    plt.show()
    
    # Plot orientation histogram with DoI value
    plt.figure(figsize=(6,4))
    plt.bar(results["Bins"][:-1], results["Orientation Histogram"], width=0.1, color='b', alpha=0.7)
    plt.xlabel("Orientation (radians)")
    plt.ylabel("Frequency")
    plt.title(f"Histogram of Preferred Orientations - {region_label}")
    plt.text(0, max(hist)*0.9, f"DoI: {doi:.3f}", fontsize=12, bbox=dict(facecolor='white', alpha=0.6))
    plt.show()
    
    return results

def batch_process_shg_images(folder_path, region_label="Region"):
    results_all = []

    for filename in os.listdir(folder_path):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.tif', '.tiff')):
            image_path = os.path.join(folder_path, filename)
            print(f"\nProcessing: {filename}")
            result = process_shg_image(image_path, region_label=filename)
            if result:
                result["Filename"] = filename
                results_all.append(result)

    return results_all


def compute_orientation_tensor(image):
    sobel_x = sobel(image, axis=0)
    sobel_y = sobel(image, axis=1)

    J11 = gaussian_filter(sobel_x**2, sigma=1)
    J22 = gaussian_filter(sobel_y**2, sigma=1)
    J12 = gaussian_filter(sobel_x * sobel_y, sigma=1)

    theta = 0.5 * np.arctan2(2 * J12, J11 - J22)
    doi_map = np.sqrt(J12**2 + (J11 - J22)**2) / (J11 + J22 + 1e-10)
    return theta, doi_map

def histogram_of_orientations(theta):
    angles = theta.flatten()
    hist, bins = np.histogram(angles, bins=36, range=(-np.pi/2, np.pi/2), density=True)
    return hist, bins

def compute_doi(hist):
    return np.std(hist) / np.max(hist) if np.max(hist) > 0 else 0

def compute_preferred_orientation(hist, bins):
    bin_centers = (bins[:-1] + bins[1:]) / 2
    preferred_angle = bin_centers[np.argmax(hist)]
    uncertainty = np.std(bin_centers, ddof=1)  # Simplified uncertainty
    return preferred_angle, uncertainty

def process_folder(folder_path):
    image_files = sorted([f for f in os.listdir(folder_path) if f.endswith(('.png', '.jpg', '.tif'))])
    dois = []
    orientations = []
    uncertainties = []
    frames = []

    for i, filename in enumerate(image_files):
        path = os.path.join(folder_path, filename)
        img = load_image(path)
        theta, _ = compute_orientation_tensor(img)
        hist, bins = histogram_of_orientations(theta)
        doi = compute_doi(hist)
        pref_ori, uncertainty = compute_preferred_orientation(hist, bins)

        dois.append(doi)
        orientations.append(pref_ori)
        uncertainties.append(uncertainty)
        frames.append(i)

    return frames, dois, orientations, uncertainties

def plot_results(frames, dois, orientations, uncertainties):
    plt.figure(figsize=(12, 5))

    # DoI Scatter Plot
    plt.subplot(1, 2, 1)
    plt.scatter(frames, dois, c='blue', label='DoI')
    plt.xlabel("Frame")
    plt.ylabel("Degree of Isotropy (DoI)")
    plt.title("DoI Evolution Over Time")
    plt.grid(True)

    # Orientation Scatter Plot
    plt.subplot(1, 2, 2)
    plt.errorbar(frames, orientations, yerr=uncertainties, fmt='o', color='green', ecolor='lightgray', capsize=3)
    plt.xlabel("Frame")
    plt.ylabel("Preferred Orientation (radians)")
    plt.title("Orientation Evolution Over Time")
    plt.grid(True)

    plt.tight_layout()
    plt.show()



results_all = batch_process_shg_images(folder_path_untreated, region_label="Anterior Stroma")
frames, dois, orientations, uncertainties = process_folder(folder_path_untreated)
plot_results(frames, dois, orientations, uncertainties)


In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt


def load_image(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        raise ValueError(f"Could not load image: {image_path}")
    return img

def normalize_image(img):
    norm_img = cv2.normalize(img.astype('float32'), None, 0, 255, cv2.NORM_MINMAX)
    return norm_img.astype(np.uint8)

def process_stack(folder_path):
    image_files = sorted([f for f in os.listdir(folder_path) if f.endswith(('.png', '.jpg', '.tif'))])
    intensities = []

    for filename in image_files:
        path = os.path.join(folder_path, filename)
        img = load_image(path)
        mean_intensity = np.mean(img)
        intensities.append(mean_intensity)

    max_intensity = max(intensities)
    normalized_intensities = [i / max_intensity for i in intensities]
    return list(range(len(normalized_intensities))), normalized_intensities

def plot_two_stacks(folder1, label1, folder2, label2):
    frames1, norm1 = process_stack(folder1)
    frames2, norm2 = process_stack(folder2)

    #Para que empiecen  desde 1
    frames1 = np.array(frames1) + 1
    frames2 = np.array(frames2) + 1


    plt.figure(figsize=(8,5))
    plt.plot(frames1, norm1, 'o-', label=label1, color='blue')
    plt.plot(frames2, norm2, 's--', label=label2, color='red')
    plt.xlabel("Frame")
    plt.ylabel("Normalized SHG Intensity")
    plt.title("Comparison of SHG Intensity")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()




def load_image(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        raise ValueError(f"Could not load image: {image_path}")
    return img

def process_stack2(folder_path):
    image_files = sorted([f for f in os.listdir(folder_path) if f.endswith(('.png', '.jpg', '.tif'))])
    intensities = []
    frames = []

    for i, filename in enumerate(image_files):
        path = os.path.join(folder_path, filename)
        img = load_image(path)
        mean_intensity = np.mean(img)
        intensities.append(mean_intensity)
        frames.append(i)

    return frames, intensities

def plot_intensities(folder1_data, folder2_data, label1, label2):
    frames1, intensities1 = folder1_data
    frames2, intensities2 = folder2_data

    #Para que empiecen  desde 1
    frames1 = np.array(frames1) + 1
    frames2 = np.array(frames2) + 1

    plt.figure(figsize=(8,5))
    plt.plot(frames1, intensities1, 'o-', label=label1, color='blue')
    plt.plot(frames2, intensities2, 'o-', label=label2, color='red')
    plt.xlabel("Frame")
    plt.ylabel("Mean SHG Intensity")
    plt.title("Mean SHG Intensity vs Frame")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

plot_two_stacks(folder_path_untreated, "Sin tratamiento", folder_path_treated, "Tras tratamiento")


# Process both stacks
untreated_data = process_stack2(folder_path_untreated)
treated_data = process_stack2(folder_path_treated)

# Plot both in the same graph


plot_intensities(untreated_data, treated_data, "Sin tratamiento", "Con tratamiento")


In [ ]:
"""Orientation analysis"""

import math
import multiprocessing
import warnings

import numba
import numba_progress
import numpy
import scipy.ndimage
from scipy.interpolate import CubicSpline
from tqdm import tqdm

gradientModes = ["finite_difference", "splines", "gaussian"]
orientationModes = ["fibre", "fiber", "membrane"]
symmetricComponents3d = numpy.array([[0, 0], [0, 1], [0, 2], [1, 1], [1, 2], [2, 2]])
symmetricComponents2d = numpy.array([[0, 0], [0, 1], [1, 1]])
nProcessesDefault = multiprocessing.cpu_count()


@numba.njit(cache=True)
def _unfoldMatrix(a):
    """
    Takes an array of length 3 or 6 and repacks it into a full symmetric matrix 2x2 or 3x3
    """
    if len(a) == 6:
        m = numpy.empty((3, 3), dtype="<f8")
        symmetricComponents = symmetricComponents3d
    elif len(a) == 3:
        m = numpy.empty((2, 2), dtype="<f8")
        symmetricComponents = symmetricComponents2d

    for n, [i, j] in enumerate(symmetricComponents):
        m[i, j] = a[n]
        # if not on diagonal fill in other side
        if i != j:
            m[j, i] = a[n]

    return m


def computeGradient(im, mode="gaussian", mask=None, anisotropy=numpy.ones(3)):
    """
    Returns the gradient of passed greylevel image.

    Parameters
    -----------
        im : array_like
            Input greyscale image, 2D or 3D

        mode : string, optional
            Selects method to compute gradients, can be either "splines", "gaussian" or "finite_difference".
            Default is "gaussian"

        anisotropy : array_like
            Relative pixel size for all axis. If your z-step is e.g. 2 times bigger than the pixel size in
            x and y, this parameter should be set to [2, 1, 1].

    Returns
    --------
        gradients : tuple of arrays
            2 or 3-component tuple of arrays (depending on 2D or 3D input)
            corresponding to (DZ) DY DX
    """
    # The sigma for the gaussian derivative, unlikely to change
    sigma = 1

    im = numpy.squeeze(im)

    twoD = im.ndim == 2

    if mode not in gradientModes:
        raise ValueError(f"{mode} not in allowable options: {gradientModes}")

    if not twoD:
        # Computing derivatives (scipy implementation truncates filter at 4 sigma).
        # 3D case
        if mode == "gaussian":
            Gx = scipy.ndimage.gaussian_filter(im, sigma, order=[0, 0, 1], mode="nearest", output=float)
            Gy = scipy.ndimage.gaussian_filter(im, sigma, order=[0, 1, 0], mode="nearest", output=float)
            Gz = scipy.ndimage.gaussian_filter(im, sigma, order=[1, 0, 0], mode="nearest", output=float)

        elif mode == "splines":
            cs_x = CubicSpline(numpy.linspace(0, im.shape[2] - 1, im.shape[2]), im, axis=2)
            Gx = cs_x(numpy.linspace(0, im.shape[2] - 1, im.shape[2]), 1)

            cs_y = CubicSpline(numpy.linspace(0, im.shape[1] - 1, im.shape[1]), im, axis=1)
            Gy = cs_y(numpy.linspace(0, im.shape[1] - 1, im.shape[1]), 1)

            cs_z = CubicSpline(numpy.linspace(0, im.shape[0] - 1, im.shape[0]), im, axis=0)
            Gz = cs_z(numpy.linspace(0, im.shape[0] - 1, im.shape[0]), 1)

        elif mode == "finite_difference":
            Gz, Gy, Gx = numpy.gradient(im)

        Gz = Gz / anisotropy[0]
        Gy = Gy / anisotropy[1]
        Gx = Gx / anisotropy[2]
        return (Gz, Gy, Gx)

    else:
        # 2D case
        if mode == "gaussian":
            Gx = scipy.ndimage.gaussian_filter(im, sigma, order=[0, 1], mode="nearest", output=float)
            Gy = scipy.ndimage.gaussian_filter(im, sigma, order=[1, 0], mode="nearest", output=float)

        elif mode == "splines":
            cs_x = CubicSpline(numpy.linspace(0, im.shape[1] - 1, im.shape[1]), im, axis=1)
            Gx = cs_x(numpy.linspace(0, im.shape[1] - 1, im.shape[1]), 1)

            cs_y = CubicSpline(numpy.linspace(0, im.shape[0] - 1, im.shape[0]), im, axis=0)
            Gy = cs_y(numpy.linspace(0, im.shape[0] - 1, im.shape[0]), 1)

        elif mode == "finite_difference":
            Gy, Gx = numpy.gradient(im)

        Gy = Gy / anisotropy[0]
        Gx = Gx / anisotropy[1]
        return (Gy, Gx)


def computeStructureTensor(gradients, sigma, mask=None):
    """
    Computes the structure tensor for every pixel of the image, averaging in a Gaussian window defined by sigma.
    Sigma is a very important parameter defining the spatial scale of interest.

    In 2D the structure tensor is a 2x2 matrix attached to each pixel and in 3D it is a 3x3 matrix, but since this tensor is symmetric
    this matrix is flattened to keep only to top-right half, and so in 2D that makes 3 components to store and
    in 3D that makes 6.
    We save in this flattened format to save memory, but also for pseudo-compatibility with skimage.feature.structure_tensor (they output a list rather than a big array)
    See https://en.wikipedia.org/wiki/Structure_tensor

    Parameters
    -----------
        gradients : tuple of array_like
            Tuple of gradient images from orientationpy.compute_gradient(im),
            This means in the 2D case a tuple of 2x2D arrays of gradients Y, X
            and in the 3D case a tuple of 3x3D arrays of Z, Y, X gradients

        sigma : float
            An integration scale giving the size over the neighbourhood in which the
            orientation is to be analysed.

        mask : boolean array, optional
            Array the same size as one of the gradients, indicating which pixels to include in the computation.

    Returns
    -------
        S : ndarray
            An array containing the computed structure tensor for each pixel.
            Output shape in 2D: 3 x Y x X
            Output shape in 2D: 6 x Z x Y x X
    """

    def multiplyAndFilter(gradients, i, j, sigma):
        return scipy.ndimage.gaussian_filter(numpy.multiply(gradients[i], gradients[j]), sigma, mode="nearest")

    if len(gradients) == 3:  # 3D
        symmetricComponents = symmetricComponents3d
    elif len(gradients) == 2:  # 2D
        symmetricComponents = symmetricComponents2d
    else:
        return None

    # Initializing structureTensor
    structureTensor = numpy.empty((len(symmetricComponents), *gradients[0].shape), dtype=float)

    # Integrating elements of structure tensor (scipy uses sequence of 1D).
    for n, [i, j] in enumerate(symmetricComponents):
        structureTensor[n] = multiplyAndFilter(gradients, i, j, sigma)

    return structureTensor


def computeGradientStructureTensor(im, sigma, mode="gaussian", anisotropy=numpy.ones(3)):
    """
    This function calls `computeGradient` with the mode and anisotropy factors passed in `mode` and `anisotropy_factors` respectively and then computes the structure tensor for each pixel (integrating in a Gaussian window of size `sigma`) with `computeStructureTensor` and returns that directly as a 3 x N x M or a 6 x N x M x O array.
    """
    # print("Computing gradients...", end="")
    g = computeGradient(im, mode=mode, anisotropy=anisotropy)
    # print("done")
    # print("Computing structure tensor...", end="")
    st = computeStructureTensor(g, sigma)
    # print("done")
    return st


def computeGradientStructureTensorBoxes(im, boxSize, mode="gaussian", anisotropy=numpy.ones(3)):
    """
    This function calls `computeGradient` with the mode and anisotropy factors passed in `mode` and `anisotropy_factors` respectively and then computes the structure tensor in touching 2/3D boxes with `computeStructureTensorBoxes` and returns the structure tensor for each box as a flattened 3 x N x M or a 6 x N x M x O array
    """
    # print("Computing gradients...", end="")
    g = computeGradient(im, mode=mode, anisotropy=anisotropy)
    # print("done")
    # print("Computing structure tensor...", end="")
    st = computeStructureTensorBoxes(g, boxSize)
    # print("done")
    return st


def computeStructureTensorBoxes(gradients, boxSize, mask=None, returnBoxCenters=False):
    """
    Computes the structure tensor in touching (no gaps and no overlaps) squares/cubes.
    This means first computing it per-pixel and then summing it in boxes.

    In 2D the structure tensor is a 2x2 matrix attached to each pixel and in 3D it is a 3x3 matrix, but since this tensor is symmetric
    this matrix is flattened to keep only to top-right half, and so in 2D that makes 3 components to store and
    in 3D that makes 6.
    We save in this flattened format to save memory, but also for pseudo-compatibility with skimage.feature.structure_tensor (they output a list rather than a big array)
    See https://en.wikipedia.org/wiki/Structure_tensor

    Parameters
    -----------
        gradients : tuple of array_like
            Tuple of gradient images from orientationpy.compute_gradient(im),
            This means in the 2D case a tuple of 2x2D arrays of gradients Y, X
            and in the 3D case a tuple of 3x3D arrays of Z, Y, X gradients

        boxSize : int or tuple
            If int, the box size in pixels in all directions.
            If tuple, should have have as many items as dimensions of the input image, and is the box size,
            in pixels in (Z), Y, X directions

        mask : boolean array, optional
            Array the same size as one of the gradients, indicating which pixels to include in the computation.

        returnBoxCenters : bool, optional
            Return the centers of the boxes?
            Optional, default = False

    Returns
    -------
        structureTensor : ndarray
            An array containing the computed structure tensor for each box.
            Output shape in 2D: 3 x Yboxes x Xboxes
            Output shape in 2D: 6 x Zboxes x Yboxes x Xboxes
    """
    if len(gradients) == 3:
        if type(boxSize) == list or type(boxSize) == tuple:
            if len(boxSize) != 3:
                print(f"computeStructureTensorBoxes(): Received 3D gradients but got len(boxSize) = {len(boxSize)}")
                return
            else:
                boxSizeZYX = boxSize
        else:
            boxSize = int(boxSize)
            boxSizeZYX = (boxSize, boxSize, boxSize)

        # Compute number of boxes, assuming top one at 0,0
        nBoxesZ = gradients[0].shape[0] // boxSizeZYX[0]
        nBoxesY = gradients[0].shape[1] // boxSizeZYX[1]
        nBoxesX = gradients[0].shape[2] // boxSizeZYX[2]

        # New empty variable to fill per-box
        structureTensorBoxes = numpy.empty((6, nBoxesZ, nBoxesY, nBoxesX))

        # Loop over boxes and integrate
        # for boxZ in tqdm(range(nBoxesZ)):
        for boxZ in range(nBoxesZ):
            for boxY in range(nBoxesY):
                for boxX in range(nBoxesX):
                    for n, [i, j] in enumerate(symmetricComponents3d):
                        # Compute i, j component of the structure tensor for all pixels in the box
                        structureTensorComponentPixelsInBox = numpy.multiply(
                            gradients[i][
                                boxZ * boxSizeZYX[0] : (boxZ + 1) * boxSizeZYX[0],
                                boxY * boxSizeZYX[1] : (boxY + 1) * boxSizeZYX[1],
                                boxX * boxSizeZYX[2] : (boxX + 1) * boxSizeZYX[2],
                            ],
                            gradients[j][
                                boxZ * boxSizeZYX[0] : (boxZ + 1) * boxSizeZYX[0],
                                boxY * boxSizeZYX[1] : (boxY + 1) * boxSizeZYX[1],
                                boxX * boxSizeZYX[2] : (boxX + 1) * boxSizeZYX[2],
                            ],
                        )

                        # Average it into the value for this box
                        structureTensorBoxes[n, boxZ, boxY, boxX] = numpy.mean(structureTensorComponentPixelsInBox)

        if returnBoxCenters:
            print("returnBoxCenters not yet implemented, just returning ST")
            return structureTensorBoxes
        else:
            return structureTensorBoxes

    elif len(gradients) == 2:
        # Check box sizes is either a two-element list or a single int
        if type(boxSize) == list or type(boxSize) == tuple:
            if len(boxSize) != 2:
                print(f"computeStructureTensorBoxes(): Received 2D gradients but got len(boxSize) = {len(boxSize)}")
                return
            else:
                boxSizeYX = boxSize
        else:
            boxSize = int(boxSize)
            boxSizeYX = (boxSize, boxSize)

        # Compute number of boxes, assuming top one at 0,0
        nBoxesY = gradients[0].shape[0] // boxSizeYX[0]
        nBoxesX = gradients[0].shape[1] // boxSizeYX[1]

        # New empty variable to fill per-box
        structureTensorBoxes = numpy.empty((3, nBoxesY, nBoxesX))

        # Loop over boxes and integrate
        for boxY in tqdm(range(nBoxesY)):
            for boxX in range(nBoxesX):
                for n, [i, j] in enumerate(symmetricComponents2d):
                    # Compute i, j component of the structure tensor for all pixels in the box
                    structureTensorComponentPixelsInBox = numpy.multiply(
                        gradients[i][
                            boxY * boxSizeYX[0] : (boxY + 1) * boxSizeYX[0],
                            boxX * boxSizeYX[1] : (boxX + 1) * boxSizeYX[1],
                        ],
                        gradients[j][
                            boxY * boxSizeYX[0] : (boxY + 1) * boxSizeYX[0],
                            boxX * boxSizeYX[1] : (boxX + 1) * boxSizeYX[1],
                        ],
                    )
                    # Average it!
                    structureTensorBoxes[n, boxY, boxX] = numpy.mean(structureTensorComponentPixelsInBox)

        if returnBoxCenters:
            print("returnBoxCenters not yet implemented, just returning ST")
            return structureTensorBoxes
        else:
            return structureTensorBoxes
    else:
        print(f"computeStructureTensorBoxes(): Unknown number of gradient dimensions: {len(gradients)}")


@numba.njit(cache=True)
def computeIntensity(structureTensor):
    """
    Computes the intesity of the gradients.
    This is the first principle invariant :math:`I_1` of the structure tensor.
    In OrientationJ this quantity is called energy.

    Parameters
    ----------
    structureTensor : tuple of numpy arrays
        The structure tensors as they are returned by
        :func:`orientationpy.main.computeStructureTensor`.

    Returns
    -------
    intentisy : numpy array
        The intensity of the gradients.
    """
    if len(structureTensor) == 6:
        # 3D
        I1 = structureTensor[0] + structureTensor[3] + structureTensor[5]
    else:
        # 2D
        I1 = structureTensor[0] + structureTensor[2]
    return I1


# @numba.njit(cache=True)
def computeStructureDirectionality(structureTensor):
    """
    Measure for how directed the structure tensor is.
    A point has no directionality but fibres and membranes do.
    This is the second main invariant :math:`J2`.

    For a mechanics interpretatin look at radial Lode coordinate.

    Parameters
    ----------
    structureTensor : tuple of numpy arrays
        The structure tensors as they are returned by
        :func:`orientationpy.main.computeStructureTensor`.

    Returns
    -------
    directionality : numpy array
        The directionality of the structure tensors.
        Zero means there is not directionality, e.g. for a point.
    """
    a = structureTensor
    if len(a) == 6:
        # 3D
        J2 = 1 / 6 * ((a[0] - a[3]) ** 2 + (a[3] - a[5]) ** 2 + (a[5] - a[0]) ** 2) + a[1] ** 2 + a[4] ** 2 + a[2] ** 2
    else:
        # 2D
        J2 = a[1] ** 2 - ((a[0] - a[2]) * (a[2] - a[0])) / 2
    return J2


@numba.njit(cache=True)
def _computeStructureType(a, I1, J2):
    """
    Numba function for computeStructureType.
    This is necessary because numba does not support
    optional arguments.
    """
    # Compute J3 as the determinant of the deviatroic tensor
    J3 = (a[0] - I1 / 3) * (a[3] - I1 / 3) * (a[5] - I1 / 3) + 2 * a[1] * a[2] * a[4] - a[2] ** 2 * (a[3] - I1 / 3) - a[1] ** 2 * (a[5] - I1 / 3) - a[4] ** 2 * (a[0] - I1 / 3)

    sin_theta_s = (J3 / 2) * (3 / J2) ** (3 / 2)

    return sin_theta_s


def computeStructureType(structureTensor, intensity=None, directionality=None):
    r"""
    Returns a value between -1 an 1 for each structure tensor, where
    -1 corresponds to a perfect fibre and 1 corresponds to a perfect plane.
    It is based on the Lode angle :math:`\theta_s` from mechanics.
    The value returned is :math:`\sin(3 \theta_s)`.

    Parameters
    ----------
    structureTensor : tuple of numpy arrays
        The structure tensors as they are returned by
        :func:`orientationpy.main.computeStructureTensor`.
    intensity : optional, numpy array
        The intensity as returned by :func:`orientationpy.main.computeIntensity`.
        If it is not provided, this function will compute it.
    directionality : optional, numpy array
        The directionality as returned by :func:`orientationpy.main.computeStructureDirectionality`.
        If it is not provided, this function will compute it.

    Returns
    -------
    structureType : numpy array
        Array of value between -1 and 1, where -1 corresponds to a fibres
        and 1 corresponds to a membrane.
    """
    if len(structureTensor) != 6:
        raise RuntimeError("The structure type is only defined in 3D.")
    if intensity is None:
        intensity = computeIntensity(structureTensor)
    if directionality is None:
        directionality = computeStructureDirectionality(structureTensor)
    structureType = _computeStructureType(structureTensor, intensity, directionality)
    mask = numpy.isclose(directionality, 0)
    if numpy.sum(mask) > 0:
        warnings.warn(
            "Some of the structure tensors provided have very low directionality. Setting their structure type to zero.",
            RuntimeWarning,
        )
        structureType[mask] = 0
    return structureType


@numba.njit(cache=True)
def _eigen_2D(arr):
    """
    This is a helper function for _eigen.
    It computes the eigen values (lambdas) and eigen vectors
    for a 2x2 matrix.
    """
    lambdas = numpy.zeros(2, dtype="<f8")
    vectors = numpy.zeros((2, 2), dtype="<f8")

    a = arr[0, 0]
    b = arr[1, 1]
    c = arr[0, 1]

    trace = a + b
    D = a * b - c**2

    lambdas[0] = trace / 2 - numpy.sqrt(trace**2 / 4 - D)
    lambdas[1] = trace / 2 + numpy.sqrt(trace**2 / 4 - D)

    vectors[0, 0] = lambdas[0] - b
    vectors[1, 0] = c
    vectors[0, 1] = lambdas[1] - b
    vectors[1, 1] = c

    return lambdas, vectors


@numba.njit(cache=True)
def _eigen(arr):
    """
    Computes the eigen values (lambdas) and eigen vectors
    for a symmetric 3x3 matrix.
    It uses closed form solutions when possible and eventaully falls
    back to numpy's eigh if non-of the closed form solutions are
    appropriate.
    It is based on an example from Pyxu's plugins
    (https://github.com/pyxu-org/cookiecutter-pyxu/blob/main/%7B%7Bcookiecutter.plugin_name%7D%7D/src/%7B%7Bcookiecutter.module_name%7D%7D/math/__init__.py)
    which is based on wikipedia:
    https://en.wikipedia.org/wiki/Eigenvalue_algorithm
    """
    lambdas = numpy.zeros(3, dtype="<f8")
    vectors = numpy.zeros((3, 3), dtype="<f8")

    a = arr[0, 0]
    d = arr[0, 1]
    f = arr[0, 2]
    b = arr[1, 1]
    e = arr[1, 2]
    c = arr[2, 2]

    p1 = d**2 + f**2 + e**2
    # Check if the case is trivial, i.e. diagonal matrix
    if p1 == 0:
        # A is diagonal.
        lambdas[0] = a
        lambdas[1] = b
        lambdas[2] = c
        vectors[0, 0] = 1
        vectors[0, 1] = 0
        vectors[0, 2] = 0
        vectors[1, 0] = 0
        vectors[1, 1] = 1
        vectors[1, 2] = 0
        vectors[2, 0] = 0
        vectors[2, 1] = 0
        vectors[2, 2] = 1
    elif f == 0 and e == 0:
        # A is an upper block matrix
        lambdas[0] = c
        vectors[:, 0] = numpy.array([0, 0, 1])
        arr2d = numpy.array([[a, d], [d, b]])
        lambdas[1:], vectors[:-1, 1:] = _eigen_2D(arr2d)
    elif d == 0 and f == 0:
        # A is a lower block matrix
        lambdas[0] = a
        vectors[:, 0] = numpy.array([1, 0, 0])
        arr2d = numpy.array([[b, e], [e, c]])
        lambdas[1:], vectors[1:, 1:] = _eigen_2D(arr2d)
    elif d == 0 and e == 0:
        # A is a "cross" matrix. This is a special case of a block
        # matrix
        lambdas[0] = b
        vectors[:, 0] = numpy.array([0, 1, 0])
        arr2d = numpy.array([[a, f], [f, c]])
        lambdas[1:], vectors2D = _eigen_2D(arr2d)
        vectors[0, 1] = vectors2D[0, 0]
        vectors[2, 1] = vectors2D[1, 0]
        vectors[0, 2] = vectors2D[0, 1]
        vectors[2, 2] = vectors2D[1, 1]
    else:
        # This part is from https://en.wikipedia.org/wiki/Eigenvalue_algorithm
        q = (a + b + c) / 3  # trace(A) is the sum of all diagonal values
        p2 = (a - q) ** 2 + (b - q) ** 2 + (c - q) ** 2 + 2 * p1
        p = (p2 / 6) ** 0.5
        det_b = (a - q) * ((b - q) * (c - q) - e**2) - d * (d * (c - q) - e * f) + f * (d * e - (b - q) * f)
        r = det_b / (2 * p**3)

        # In exact arithmetic for a symmetric matrix -1 <= r <= 1
        # but computation error can leave it slightly lambdasside this range.
        if r <= -1:
            phi = numpy.pi / 3
        elif r >= 1:
            phi = 0
        else:
            phi = numpy.arccos(r) / 3

        # the eigenvalues satisfy lambdas[2] <= lambdas[1] <= lambdas[0]
        lambdas[0] = q + 2 * p * numpy.cos(phi + (2 * numpy.pi / 3))
        lambdas[2] = q + 2 * p * numpy.cos(phi)
        lambdas[1] = 3 * q - lambdas[0] - lambdas[2]  # since trace(A) = eig1 + eig2 + eig3

        # Compute the eigen vectors
        for i in range(3):
            if not numpy.isclose((f * (b - lambdas[i]) - d * e), 0):
                m = (d * (c - lambdas[i]) - e * f) / (f * (b - lambdas[i]) - d * e)
                vectors[0, i] = (lambdas[i] - c - e * m) / f
                vectors[1, i] = m
                vectors[2, i] = 1
            elif not numpy.isclose((e * (a - lambdas[i]) - d * f), 0):
                m = (f * (b - lambdas[i]) - d * e) / (e * (a - lambdas[i]) - d * f)
                vectors[0, i] = m
                vectors[1, i] = 1
                vectors[2, i] = (lambdas[i] - b - d * m) / e
            elif not numpy.isclose((d * (c - lambdas[i]) - f * e), 0):
                m = (e * (a - lambdas[i]) - f * d) / (d * (c - lambdas[i]) - f * e)
                vectors[0, i] = 1
                vectors[1, i] = (lambdas[i] - a - f * m) / d
                vectors[2, i] = m
            else:
                lambdas, vectors = numpy.linalg.eigh(arr)

    # Make unit vectors
    vectors[:, 0] /= numpy.linalg.norm(vectors[:, 0])
    vectors[:, 1] /= numpy.linalg.norm(vectors[:, 1])
    vectors[:, 2] /= numpy.linalg.norm(vectors[:, 2])

    # Sort by eigenvalues
    sort_indices = numpy.argsort(lambdas)
    lambdas = lambdas[sort_indices]
    vectors = vectors[:, sort_indices]

    return lambdas, vectors


@numba.njit(parallel=True, cache=True)
def orientationFunction(
    structureTensor,
    progressProxy,
    fibre=True,
):
    theta = numpy.zeros(structureTensor.shape[1:], dtype="<f8")
    phi = numpy.zeros(structureTensor.shape[1:], dtype="<f8")

    for z in numba.prange(0, structureTensor.shape[1]):
        for y in range(0, structureTensor.shape[2]):
            for x in range(0, structureTensor.shape[3]):
                g = _unfoldMatrix(structureTensor[:, z, y, x])
                w, v = _eigen(g)

                if not fibre:
                    m = numpy.argmax(w)
                else:  # (mode == "fibre")
                    m = numpy.argmin(w)

                selectedEigenVector = v[:, m]

                # Flip over -z
                if selectedEigenVector[0] < 0:
                    selectedEigenVector *= -1

                # polar angle
                theta[z, y, x] = numpy.rad2deg(math.acos(numpy.abs(selectedEigenVector[0])))
                # azimuthal angle
                phi[z, y, x] = numpy.rad2deg(math.atan2(selectedEigenVector[1], selectedEigenVector[2]))
                if phi[z, y, x] < 0:
                    phi[z, y, x] += 360
                elif phi[z, y, x] >= 360:
                    phi[z, y, x] -= 360

        progressProxy.update(1)
    return theta, phi


def computeOrientation(
    structureTensor,
    mode="fibre",
    nProcesses=nProcessesDefault,
):
    """
    Takes in a pre-computed field of Structure Tensors and returns orientations.

    Parameters
    -----------
        structureTensor : numpy array
            2D or 3D structureTensor array from computeStructureTensor() or computeStructureTensorBoxes()

        mode : string, optional
            What mode to use for orientations -- N.B., this is only relevant in 3D.
            Are you looking for a "fibre" (1D object) or "membrane" (2D object)?
            Default = "fibre"

    Returns
    --------
        A dictionary containing:
          - theta
          - phi (only for 3D data)
    """
    outputDict = {}

    if mode not in orientationModes:
        raise ValueError(f"orientation mode {mode} not in supported modes: {orientationModes}")

    if len(structureTensor.shape) == 4:
        # We're in 3D!
        assert structureTensor.shape[0] == 6

        with numba_progress.ProgressBar(total=structureTensor.shape[1]) as progress:
            theta, phi = orientationFunction(
                structureTensor,
                progress,
                fibre=not (mode == "membrane"),
            )

        outputDict["theta"] = theta
        outputDict["phi"] = phi

        return outputDict

    elif len(structureTensor.shape) == 3:
        # We're in 2D!
        assert structureTensor.shape[0] == 3

        if mode == "membrane":
            raise ValueError(f"membrane doesn't exist in 2D")

        outputDict["theta"] = numpy.rad2deg(
            1
            / 2
            * numpy.arctan2(
                structureTensor[1, :, :] + structureTensor[1, :, :],
                structureTensor[0, :, :] - structureTensor[2, :, :],
            )
        )

        return outputDict

    else:
        raise ValueError(f"structure tensor has unexpected shape {len(structureTensor)}, should be 3 for 2D and 4 for 3D")


def anglesToVectors(orientations):
    """
    Takes in angles in degrees and returns corresponding unit vectors in Z Y X.

    Parameters
    ----------
        orientations : dictionary
            Dictionary containing a numpy array of  'theta' in degrees in 2D,
            and also a numpy array of 'phi' if 3D.

    Returns
    -------
        unitVectors : 2 or 3 x N x M (x O)
            YX or ZYX unit vectors
    """
    if type(orientations) is not dict:
        raise TypeError(f"dictionary with 'theta' (and optionally 'phi') key needed, you passed a {type(orientations)}")

    if "phi" in orientations.keys():
        # print("orientationAngleToOrientationVector(): 3D!")
        theta = numpy.deg2rad(orientations["theta"])
        phi = numpy.deg2rad(orientations["phi"])
        coordsZYX = numpy.zeros(
            (
                3,
                theta.shape[0],
                theta.shape[1],
                theta.shape[2],
            )
        )
        coordsZYX[2, :, :] = numpy.sin(theta) * numpy.cos(phi)
        coordsZYX[1, :, :] = numpy.sin(theta) * numpy.sin(phi)
        coordsZYX[0, :, :] = numpy.cos(theta)

        return coordsZYX
    # if type(orientations) == numpy.array:
    elif "theta" in orientations.keys():
        # print("orientationAngleToOrientationVector(): 2D!")
        # 2D case
        coordsYX = numpy.zeros(
            (
                2,
                orientations["theta"].shape[0],
                orientations["theta"].shape[1],
            )
        )

        coordsYX[0] = -numpy.sin(numpy.deg2rad(orientations["theta"]))
        coordsYX[1] = numpy.cos(numpy.deg2rad(orientations["theta"]))

        return coordsYX

    else:
        raise KeyError("couldn't find 'theta' (and optionally 'phi') key in passed orientations")


def _decomposeStructureTensor(structureTensor):
    """
    Returns the structure tensor of input structure tensor.
    Note: this function only works for 3D and 2D image data

    Parameters
    -----------
        structure_tensor : Array of shape (2/3, 2/3, ...)
            The second moment matrix of the im that you want to calculate
            the orientation of.

    Returns
    -------
        eigenvalues : Array of shape (..., M)
            The eigenvalues, each repeated according to its multiplicity.
            The eigenvalues are not necessarily ordered. The resulting
            array will be of complex type, unless the imaginary part is
            zero in which case it will be cast to a real type. When `structure_tensor`
            is real the resulting eigenvalues will be real (0 imaginary
            part) or occur in conjugate pairs

        eigenvectors : (..., M, M) array
            The normalized (unit "length") eigenvectors, such that the
            column ``eigenvectors[:,i]`` is the eigenvector corresponding to the
            eigenvalue ``eigenvalues[i]``.

    Notes
    -----
        Built upon the numpy.linalg.eig function, for more information:
        https://github.com/numpy/numpy/blob/v1.23.0/numpy/linalg/linalg.py#L1182-L1328
    """

    # 2D
    if len(structureTensor.shape) == 3:
        assert structureTensor.shape[0] == 3
        # Initializing eigen images
        eigenvectors = numpy.zeros(shape=(2, 2, structureTensor.shape[1], structureTensor.shape[2]))
        eigenvalues = numpy.zeros(shape=(2, structureTensor.shape[1], structureTensor.shape[2]))
        for y in range(0, structureTensor.shape[1]):
            for x in range(0, structureTensor.shape[2]):
                eigenvalues[:, y, x], eigenvectors[:, :, y, x] = numpy.linalg.eig(_unfoldMatrix(structureTensor[:, y, x]))
    # else:
    # print(f"_decomposeStructureTensor(): Passed structure tensor has {len(structureTensor.shape)} dimensions, don't know what to do")

    return eigenvalues, eigenvectors







#Mio que es para tener algo cuantitativo de la orientacion

def computeCoherenceMap(structure_tensor):
    """
    Calcula el mapa de coherencia (orientación bien definida) para un tensor de estructura
    ya calculado, en 2D (3 componentes) o 3D (6 componentes).

    Parámetros
    ----------
    structure_tensor : ndarray
        Array de tensores de estructura con shape:
        - 3 x Y x X (2D)
        - 6 x Z x Y x X (3D)

    Retorna
    -------
    coherence_map : ndarray
        Mapa de coherencia con misma shape espacial que el tensor (sin la primera dimensión).
    """
    import numpy as np

    dims = structure_tensor.shape[0]
    
    if dims == 3:
        # 2D case
        _, Y, X = structure_tensor.shape
        coherence = np.zeros((Y, X), dtype=np.float32)

        for y in range(Y):
            for x in range(X):
                tensor = _unfoldMatrix(structure_tensor[:, y, x])
                eigvals = np.linalg.eigvalsh(tensor)
                eigvals = np.sort(eigvals)[::-1]
                l1, l2 = eigvals[0], eigvals[1]
                if l1 + l2 > 0:
                    coherence[y, x] = (l1 - l2) / (l1 + l2)
                else:
                    coherence[y, x] = 0.0

        return coherence

    elif dims == 6:
        # 3D case
        _, Z, Y, X = structure_tensor.shape
        coherence = np.zeros((Z, Y, X), dtype=np.float32)

        for z in range(Z):
            for y in range(Y):
                for x in range(X):
                    tensor = _unfoldMatrix(structure_tensor[:, z, y, x])
                    eigvals = np.linalg.eigvalsh(tensor)
                    eigvals = np.sort(eigvals)[::-1]
                    l1, l2, l3 = eigvals
                    denom = l1 + l2 + l3
                    if denom > 0:
                        coherence[z, y, x] = (l1 - l2) / denom  # Coherencia lineal
                    else:
                        coherence[z, y, x] = 0.0

        return coherence

    else:
        raise ValueError(f"Dimensión del tensor inesperada: {dims}")

def compute_orientation_metrics(structureTensor):
    """
    Compute quantitative orientation metrics per layer and globally
    
    Returns:
        metrics: dict with keys:
            'coherence_map' : ndarray (coherencia por caja)
            'mean_coherence_per_layer' : list (coherencia media por capa)
            'global_mean_coherence' : float (coherencia media global)
    """
    coherence_map = computeCoherenceMap(structureTensor)
    
    # Coherencia media por capa (promedio en XY para cada Z)
    if coherence_map.ndim == 3:  # 3D
        mean_coherence_per_layer = numpy.mean(coherence_map, axis=(1, 2))
    else:  # 2D
        mean_coherence_per_layer = [numpy.mean(coherence_map)]
    
    # Coherencia media global
    global_mean_coherence = numpy.mean(coherence_map)
    
    return {
        'coherence_map': coherence_map,
        'mean_coherence_per_layer': mean_coherence_per_layer,
        'global_mean_coherence': global_mean_coherence
    }



#Esto no estoy seguro de que sirva, pero lo dejo por si acaso

# ...existing code...
def order_coefficient(Mi):
    """
    Calcula el Order Coefficient según la fórmula proporcionada.
    Mi: array con el número de elementos alineados o intensidad por caja.
    """
    Mi = np.array(Mi)
    N_windows = len(Mi)
    M_total = np.sum(Mi)
    if M_total == 0 or N_windows < 2:
        return 0.0

    numerador = np.sum((Mi / M_total - 1 / N_windows) ** 2)
    den1 = 2 * ( (1/2) - (1/N_windows) )**2
    den2 = (N_windows - 2) * ( (1/N_windows)**2 )
    denominador = den1 + den2

    return numerador / denominador






In [ ]:
"""
Library of SPAM functions for plotting orientations in 3D
Copyright (C) 2020 SPAM Contributors

This program is free software: you can redistribute it and/or modify it
under the terms of the GNU General Public License as published by the Free
Software Foundation, either version 3 of the License, or (at your option)
any later version.

This program is distributed in the hope that it will be useful, but WITHOUT
ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or
FITNESS FOR A PARTICULAR PURPOSE.  See the GNU General Public License for
more details.

You should have received a copy of the GNU General Public License along with
this program.  If not, see <http://www.gnu.org/licenses/>.
"""


#
# Edward Ando 17/11/2011
#
# Attempt at programming rose plots myself.

# Assuming that the contacts are kind of flat, and so the normal vector pointing up from them is correct
# which means using Visilog's Orientation 2 vectors.

# Modified in order to allow vector components to be taken as input directly

# 2012.08.27 - adding ability to output projected components, in order to allow plotting
#   with gnuplot

# Completely new version with objective of:
#   - reading in different formats (3D coordinates (x,y,z,), Spherical Coordinates, Cylindrical)
#   - outputting any other format
#   - Different projections onto the plane (Lambert, Stereo, direct, etc...)
#   - Possibility to colour the negative part of the projected sphere in a different colour
#   - Projection point-by-point or binned with Hugues Talbot and Clara's code, which
#       gives the very convenient cutting of the circle into in equal area parts:
#         Jaquet, C., Andò, E., Viggiani, G., & Talbot, H. (2013).
#         Estimation of separating planes between touching 3D objects using power watershed.
#         In Mathematical Morphology and Its Applications to Signal and Image Processing (pp. 452-463).
#         Springer Berlin Heidelberg.

# Internal data format will be x,y,z sphere, with x-y defining the plane of projection.

# 2015-07-24 -- EA and MW -- checking everything, there were many bugs -- the points were only plotted to an extent of 1 and not radiusMax
#               created plotting function, which allows radius labels to be updated.

# 2016-11-08 -- MW -- binning was still erroneous. an angularBin (lines 348ff) was usually put to the next higher bin, so not rounded correctly
#               rounding now with numpy.rint and if the orientation doesn't belong to the last bin, it has to be put in the first one
#                 as the first bin extends to both sides of 0 Degrees. -> check validation example in lines 227ff

# 2017-06-01 -- MW -- updating for the current numpy version 1.12.1 and upgrading matplotlib to 2.0.2

# 2017-06-26 -- MW -- there still was a small bug in the binning: for the angular bin numpy.rint was used -- replaced it with numpy.floor (line 375)
#                     benchmarking points are in lines 217ff.

# 2018-02-19 -- EA -- changes in the new matplot version revealed a problem. Eddy solved it for the spam client, MW modified this one here.

# 2018-04-20 -- MW -- adding relative bin counts -- to normalise the bincounts by the average overall bin count.
#                     enables to plot many states with the same legend!

import math
import multiprocessing
import os
import random
import sys

import matplotlib
import matplotlib.colors as mcolors
import matplotlib.pyplot
import numpy
import orientationpy
from mpl_toolkits.mplot3d.art3d import Line3DCollection, Poly3DCollection

nProcessesDefault = multiprocessing.cpu_count()

VERBOSE = False

# ask numpy to print 0.000 without scientific notation
numpy.set_printoptions(suppress=True)
numpy.set_printoptions(precision=3)




def projectOrientations3d(vector, coordSystem, projectionSystem):
    """
    This functions projects a 3D vector from a given coordinate system into a 2D plane given by a defined projection.

    Parameters
    ----------
        vector: 1x3 array of floats
            Vector to be projected
            For cartesian system: ZYX
            For spherical system: r, theta (inclination), phi (azimuth) in Radians

        coordSystem: string
            Coordinate system of the vector
            Either "cartesian" or "spherical"

        projectionSystem : string
            Projection to be used
            Either "lambert", "stereo" or "equidistant"

    Returns
    -------
        projection_xy: 1x2 array of floats
            X and Y coordinates of the projected vector

        projection_theta_r: 1x2 array of floats
            Theta and R coordinates of the projected vector in radians

    """

    projection_xy_local = numpy.zeros(2)
    projection_theta_r_local = numpy.zeros(2)

    # Reshape the vector and check for errors in shape
    try:
        vector = numpy.reshape(vector, (3, 1))
    except:
        print("\n projectOrientations3d: The vector must be an array of 1x3")
        return

    if coordSystem == "spherical":
        # unpack vector

        r, theta, phi = vector

        x = r * math.sin(theta) * math.cos(phi)
        y = r * math.sin(theta) * math.sin(phi)
        z = r * math.cos(theta)

    elif coordSystem == "cartesian":

        # unpack vector
        z, y, x = vector
        # we're in cartesian coordinates, (x-y-z mode) Calculate spherical coordinates
        # passing to 3d spherical coordinates too...
        # From: https://en.wikipedia.org/wiki/Spherical_coordinate_system
        #  Several different conventions exist for representing the three coordinates, and for the order in which they should be written.
        #  The use of (r, θ, φ) to denote radial distance, inclination (or elevation), and azimuth, respectively, is common practice in physics, and is specified by ISO standard 80000-2 :2009, and earlier in ISO 31-11 (1992).
        r = numpy.sqrt(x**2 + y**2 + z**2)
        theta = math.acos(z / r)  # inclination
        phi = math.atan2(y, x)  # azimuth

    else:
        print("\n projectOrientations3d: Wrong coordinate system")
        return

    if projectionSystem == "lambert":  # dividing by sqrt(2) so that we're projecting onto a unit circle
        projection_xy_local[0] = x * (math.sqrt(2 / (1 + z)))
        projection_xy_local[1] = y * (math.sqrt(2 / (1 + z)))

        # sperhical coordinates -- CAREFUL as per this wikipedia page: https://en.wikipedia.org/wiki/Lambert_azimuthal_equal-area_projection
        #   the symbols for inclination and azimuth ARE INVERTED WITH RESPEST TO THE SPHERICAL COORDS!!!
        projection_theta_r_local[0] = phi
        #                                                  HACK: doing math.pi - angle in order for the +z to be projected to 0,0
        projection_theta_r_local[1] = 2 * math.cos((math.pi - theta) / 2)

        # cylindrical coordinates
        # projection_theta_r_local[0] = phi
        # projection_theta_r_local[1] = math.sqrt( 2.0 * ( 1 + z ) )

    elif projectionSystem == "stereo":
        projection_xy_local[0] = x / (1 - z)
        projection_xy_local[1] = y / (1 - z)

        # https://en.wikipedia.org/wiki/Stereographic_projection uses a different standard from the page on spherical coord Spherical_coordinate_system
        projection_theta_r_local[0] = phi
        #                                        HACK: doing math.pi - angle in order for the +z to be projected to 0,0
        #                                                                             HACK: doing math.pi - angle in order for the +z to be projected to 0,0
        projection_theta_r_local[1] = numpy.sin(math.pi - theta) / (1 - numpy.cos(math.pi - theta))

    elif projectionSystem == "equidistant":
        # https://en.wikipedia.org/wiki/Azimuthal_equidistant_projection
        # TODO: To be checked, but this looks like it should -- a straight down projection.
        projection_xy_local[0] = math.sin(phi)
        projection_xy_local[1] = math.cos(phi)

        projection_theta_r_local[0] = phi
        projection_theta_r_local[1] = numpy.cos(theta - math.pi / 2)

    else:
        print("\n projectOrientations3d: Wrong projection system")
        return

    return projection_xy_local, projection_theta_r_local








import matplotlib

VERBOSE = False


def plotOrientations3d(
    orientations_zyx,
    projection="lambert",
    plot="both",
    binValueMin=None,
    binValueMax=None,
    binNormalisation=False,
    numberOfRings=9,
    pointMarkerSize=8,
    cmap=matplotlib.pyplot.cm.RdBu_r,
    title="",
    subtitle={"points": "", "bins": ""},
    saveFigPath=None,
):
    """
    Main function for plotting 3D orientations.
    This function plots orientations (described by unit-direction vectors) from a sphere onto a plane.

    One useful trick for evaluating these orientations is to project them with a "Lambert equal area projection", which means that an isotropic distribution of angles is projected as equally filling the projected space.

    Parameters
    ----------
        orientations : Nx3 numpy array of floats
            Z, Y and X components of direction vectors.
            Non-unit vectors are normalised.

        projection : string, optional
            Selects different projection modes:
                **lambert** : Equal-area projection, default and highly reccommended. See https://en.wikipedia.org/wiki/Lambert_azimuthal_equal-area_projection

                **equidistant** : equidistant projection

        plot : string, optional
            Selects which plots to show:
                **points** : shows projected points individually
                **bins** : shows binned orientations with counts inside each bin as colour
                **both** : shows both representations side-by-side, default

        title : string, optional
            Plot main title. Default = ""

        subtitle : dictionary, optional
            Sub-plot titles:
                **points** : Title for points plot. Default = ""
                **bins** : Title for bins plot. Default = ""

        binValueMin : int, optional
            Minimum colour-bar limits for bin view.
            Default = None (`i.e.`, auto-set)

        binValueMax : int, optional
            Maxmum colour-bar limits for bin view.
            Default = None (`i.e.`, auto-set)

        binNormalisation : bool, optional
            In binning mode, should bin counts be normalised by mean counts on all bins
            or absolute counts?

        cmap : matplotlib colour map, optional
            Colourmap for number of counts in each bin in the bin view.
            Default = ``matplotlib.pyplot.cm.RdBu_r``

        numberOfRings : int, optional
            Number of rings (`i.e.`, radial bins) for the bin view.
            The other bins are set automatically to have uniform sized bins using an algorithm from Jacquet and Tabot.
            Default = 9 (quite small bins)

        pointMarkerSize : int, optional
            Size of points in point view (5 OK for many points, 25 good for few points/debugging).
            Default = 8 (quite big points)

        saveFigPath : string, optional
            Path to save figure to -- stops the graphical plotting.
            Default = None

    Returns
    -------
        None -- A matplotlib graph is created and show()n

    Note
    ----
        Authors: Edward Andò, Hugues Talbot, Clara Jacquet and Max Wiebicke
    """
    import matplotlib.pyplot

    # ========================================================================
    # ==== Reading in data, and formatting to x,y,z sphere                 ===
    # ========================================================================
    numberOfPoints = orientations_zyx.shape[0]

    # ========================================================================
    # ==== Check that all the vectors are unit vectors                     ===
    # ========================================================================
    if VERBOSE:
        print("\t-> Normalising all vectors in x-y-z representation..."),

    # from http://stackoverflow.com/questions/2850743/numpy-how-to-quickly-normalize-many-vectors
    norms = numpy.apply_along_axis(numpy.linalg.norm, 1, orientations_zyx)
    orientations_zyx = orientations_zyx / norms.reshape(-1, 1)

    if VERBOSE:
        print("done.")

    # ========================================================================
    # ==== At this point we should have clean x,y,z data in memory         ===
    # ========================================================================
    if VERBOSE:
        print("\t-> We have %i orientations in memory." % (numberOfPoints))

    # Since this is the final number of vectors, at this point we can set up the
    #   matrices for the projection.
    projection_xy = numpy.zeros((numberOfPoints, 2))

    # TODO: Check if there are any values less than zero or more that 2*pi
    projection_theta_r = numpy.zeros((numberOfPoints, 2))

    # ========================================================================
    # ==== Projecting from x,y,z sphere to the desired projection          ===
    # ========================================================================
    # TODO: Vectorise this...
    for vectorN in range(numberOfPoints):
        # unpack 3D x,y,z
        z, y, x = orientations_zyx[vectorN]
        # print "\t\txyz = ", x, y, z

        # fold over the negative half of the sphere
        #     flip every component of the vector over
        if z < 0:
            z = -z
            y = -y
            x = -x

        projection_xy[vectorN], projection_theta_r[vectorN] = projectOrientations3d([z, y, x], "cartesian", projection)

    # get radiusMax based on projection
    #                                    This is only limited to sqrt(2) because we're flipping over the negative side of the sphere
    if projection == "lambert":
        radiusMax = numpy.sqrt(2)
    elif projection == "stereo":
        radiusMax = 1.0
    elif projection == "equidistant":
        radiusMax = 1.0

    if VERBOSE:
        print("\t-> Biggest projected radius (r,t) = {}".format(numpy.abs(projection_theta_r[:, 1]).max()))

    # print "projection_xy\n", projection_xy
    # print "\n\nprojection_theta_r\n", projection_theta_r

    if plot == "points" or plot == "both":
        fig = matplotlib.pyplot.figure()
        fig.suptitle(title)
        if plot == "both":
            ax = fig.add_subplot(121, polar=True)
        else:
            ax = fig.add_subplot(111, polar=True)

        ax.set_title(subtitle["points"] + "\n")

        # set the line along which the numbers are plotted to 0°
        # ax.set_rlabel_position(0)
        matplotlib.pyplot.axis((0, math.pi * 2, 0, radiusMax))

        # set radius grids to 15, 30, etc, which means 6 numbers (r=0 not included)
        radiusGridAngles = numpy.arange(15, 91, 15)
        radiusGridValues = []
        for angle in radiusGridAngles:
            #                        - project the 15, 30, 45 as spherical coords, and select the r part of theta r-
            #               - append to list of radii -

            radiusGridValues.append(projectOrientations3d([0, angle * math.pi / 180.0, 1], "spherical", projection)[1][1])
        #                                       --- list comprehension to print 15°, 30°, 45° ----------
        ax.set_rgrids(radiusGridValues, labels=[r"%02i$^\circ$" % (x) for x in numpy.arange(15, 91, 15)], angle=None, fmt=None)
        ax.plot(projection_theta_r[:, 0], projection_theta_r[:, 1], ".", markersize=pointMarkerSize)

        if plot == "points":
            matplotlib.pyplot.show()

    if plot == "bins" or plot == "both":
        # ========================================================================
        # ==== Binning the data -- this could be optional...                   ===
        # ========================================================================
        # This code inspired from Hugues Talbot and Clara Jaquet's developments.
        # As published in:
        #   Identifying and following particle-to-particle contacts in real granular media: an experimental challenge
        #   Gioacchino Viggiani, Edward Andò, Clara Jaquet and Hugues Talbot
        #   Keynote Lecture
        #   Particles and Grains 2013 Sydney
        #
        # ...The number of radial bins (numberOfRings)
        # defines the radial binning, and for each radial bin starting from the centre,
        # the number of angular bins is  4(2n + 1)
        #
        import matplotlib.collections

        # from matplotlib.colors import Normalize
        import matplotlib.colorbar
        import matplotlib.patches

        if plot == "both":
            ax = fig.add_subplot(122, polar=True)
        if plot == "bins":
            fig = matplotlib.pyplot.figure()
            ax = fig.add_subplot(111, polar=True)

        if VERBOSE:
            print("\t-> Starting Data binning...")

        # This must be an integer -- could well be a parameter if this becomes a function.
        if VERBOSE:
            print("\t-> Number of Rings (radial bins) = ", numberOfRings)

        # As per the publication, the maximum number of bins for each ring, coming from the inside out is 4(2n + 1):
        numberOfAngularBinsPerRing = numpy.arange(1, numberOfRings + 1, 1)
        numberOfAngularBinsPerRing = 4 * (2 * numberOfAngularBinsPerRing - 1)

        if VERBOSE:
            print("\t-> Number of angular bins per ring = ", numberOfAngularBinsPerRing)

        # defining an array with dimensions numberOfRings x numberOfAngularBinsPerRing
        binCounts = numpy.zeros((numberOfRings, numberOfAngularBinsPerRing[-1]))

        # ========================================================================
        # ==== Start counting the vectors into bins                            ===
        # ========================================================================
        for vectorN in range(numberOfPoints):
            # unpack projected angle and radius for this point
            angle, radius = projection_theta_r[vectorN, :]

            # Flip over negative angles
            if angle < 0:
                angle += 2 * math.pi
            if angle > 2 * math.pi:
                angle -= 2 * math.pi

            # Calculate right ring number
            ringNumber = int(numpy.floor(radius / (radiusMax / float(numberOfRings))))

            # Check for overflow
            if ringNumber > numberOfRings - 1:
                if VERBOSE:
                    print("\t-> Point with projected radius = {:f} is a problem (radiusMax = {:f}), putting in furthest  bin".format(radius, radiusMax))
                ringNumber = numberOfRings - 1

            # Calculate the angular bin
            angularBin = int(numpy.floor((angle) / (2 * math.pi / float(numberOfAngularBinsPerRing[ringNumber])))) + 1

            # print "numberOfAngularBinsPerRing", numberOfAngularBinsPerRing[ringNumber] - 1
            # Check for overflow
            #  in case it doesn't belong in the last angularBin, it has to be put in the first one!
            if angularBin > numberOfAngularBinsPerRing[ringNumber] - 1:
                if VERBOSE:
                    print("\t-> Point with projected angle = %f does not belong to the last bin, putting in first bin" % (angle))
                angularBin = 0

            # now that we know what ring, and angular bin you're in add one count!
            binCounts[ringNumber, angularBin] += 1

        # ========================================================================
        # === Plotting binned data                                             ===
        # ========================================================================

        plottingRadii = numpy.linspace(radiusMax / float(numberOfRings), radiusMax, numberOfRings)
        # print "Plotting radii:", plottingRadii

        # ax  = fig.add_subplot(122, polar=True)
        # matplotlib.pyplot.axis()
        # ax = fig.add_axes([0.1, 0.1, 0.8, 0.8], polar=True)
        bars = []

        # add two fake, small circles at the beginning so that they are overwritten
        #   they will be coloured with the min and max colour
        #              theta   radius    width
        bars.append([0, radiusMax, 2 * math.pi])
        bars.append([0, radiusMax, 2 * math.pi])
        # bars.append(ax.bar(0,   radiusMax,    2*math.pi, bottom=0.0))
        # bars.append(ax.bar(0,   radiusMax,    2*math.pi, bottom=0.0))

        # --- flatifiying binned data for colouring wedges                    ===
        flatBinCounts = numpy.zeros(numpy.sum(numberOfAngularBinsPerRing) + 2)

        # Bin number as we go through the bins to add the counts in order to the flatBinCounts
        # This is two in order to skip the first to fake bins which set the colour bar.
        binNumber = 2

        # --- Plotting binned data, from the outside, inwards.                 ===
        if binNormalisation:
            avg_binCount = float(numberOfPoints) / numpy.sum(numberOfAngularBinsPerRing)
            # print "\t-> Number of points = ", numberOfPoints
            # print "\t-> Number of bins   = ", numpy.sum(numberOfAngularBinsPerRing)
            if VERBOSE:
                print("\t-> Average binCount = ", avg_binCount)

        for ringNumber in range(numberOfRings)[::-1]:
            deltaTheta = 360 / float(numberOfAngularBinsPerRing[ringNumber])
            deltaThetaRad = 2 * math.pi / float(numberOfAngularBinsPerRing[ringNumber])

            # --- Angular bins                                                 ---
            for angularBin in range(numberOfAngularBinsPerRing[ringNumber]):
                # ...or add bars
                #                           theta                             radius                  width
                bars.append([angularBin * deltaThetaRad - deltaThetaRad / 2.0, plottingRadii[ringNumber], deltaThetaRad])
                # bars.append(ax.bar(angularBin*deltaThetaRad - deltaThetaRad/2.0, plottingRadii[ ringNumber ], deltaThetaRad, bottom=0.0))

                # Add the number of vectors counted for this bin
                if binNormalisation:
                    flatBinCounts[binNumber] = binCounts[ringNumber, angularBin] / avg_binCount
                else:
                    flatBinCounts[binNumber] = binCounts[ringNumber, angularBin]

                # Add one to bin number
                binNumber += 1

        del binNumber

        # figure out auto values if they're requested.
        if binValueMin is None:
            binValueMin = flatBinCounts[2::].min()
        if binValueMax is None:
            binValueMax = flatBinCounts[2::].max()

        # Add two flat values for the initial wedges.
        flatBinCounts[0] = binValueMin
        flatBinCounts[1] = binValueMax

        ##                           theta                   radius                          width
        barsPlot = ax.bar(numpy.array(bars)[:, 0], numpy.array(bars)[:, 1], width=numpy.array(bars)[:, 2], bottom=0.0)

        for binCount, bar in zip(flatBinCounts, barsPlot):
            bar.set_facecolor(cmap((binCount - binValueMin) / float(binValueMax - binValueMin)))

        # matplotlib.pyplot.axis([ 0, radiusMax, 0, radiusMax ])
        matplotlib.pyplot.axis([0, numpy.deg2rad(360), 0, radiusMax])

        # colorbar = matplotlib.pyplot.colorbar(barsPlot, norm=matplotlib.colors.Normalize(vmin=minBinValue, vmax=maxBinValue))
        # Set the colormap and norm to correspond to the data for which
        # the colorbar will be used.

        norm = matplotlib.colors.Normalize(vmin=binValueMin, vmax=binValueMax)

        # ColorbarBase derives from ScalarMappable and puts a colorbar
        # in a specified axes, so it has everything needed for a
        # standalone colorbar.  There are many more kwargs, but the
        # following gives a basic continuous colorbar with ticks
        # and labels.
        ax3 = fig.add_axes([0.9, 0.1, 0.03, 0.8])
        cb1 = matplotlib.colorbar.ColorbarBase(ax3, cmap=cmap, norm=norm, label="Number of vectors in bin")

        # set the line along which the numbers are plotted to 0°
        # ax.set_rlabel_position(0)

        # set radius grids to 15, 30, etc, which means 6 numbers (r=0 not included)
        radiusGridAngles = numpy.arange(15, 91, 15)
        radiusGridValues = []
        for angle in radiusGridAngles:
            #                        - project the 15, 30, 45 as spherical coords, and select the r part of theta r-
            #               - append to list of radii -
            radiusGridValues.append(projectOrientations3d([0, angle * math.pi / 180.0, 1], "spherical", projection)[1][1])
        #                                       --- list comprehension to print 15°, 30°, 45° ----------
        ax.set_rgrids(radiusGridValues, labels=[r"%02i$^\circ$" % (x) for x in numpy.arange(15, 91, 15)], angle=None, fmt=None)

        fig.subplots_adjust(left=0.05, right=0.85)
        # cb1.set_label('Some Units')
        
        
        # === 1) Radial + Angular binning ===
        angBins = 4 * (2 * numpy.arange(1, numberOfRings + 1) - 1)
        binCounts = numpy.zeros((numberOfRings, angBins[-1]), dtype=int)
        for angle, radius in projection_theta_r:
            a = angle % (2 * math.pi)
            ring = min(int(numpy.floor(radius / (radiusMax / numberOfRings))), numberOfRings - 1)
            bins = angBins[ring]
            idx = int(numpy.floor(a / (2 * math.pi / bins)))
            binCounts[ring, idx if idx < bins else 0] += 1
        Mi = binCounts.flatten()
        oc = order_coefficient(Mi)

        # === 2) Angular-only binning ===
        nbins = 36  # 10° windows
        counts_ang = numpy.zeros(nbins, dtype=int)
        width = 2 * math.pi / nbins
        for angle in projection_theta_r[:, 0] % (2 * math.pi):
            bi = int(angle // width)
            counts_ang[bi] += 1
        oc_ang = order_coefficient(counts_ang)
        # Cuadro con los coeficientes
        textstr = f"Coef. Orden radial+angular: {oc:.3f}\nCoef. Orden angular: {oc_ang:.3f}"
        props = dict(boxstyle='round', facecolor='wheat', alpha=0.7)
        ax.text(-1.55, -0.45, textstr, transform=ax.transAxes, fontsize=13,
                verticalalignment='bottom', horizontalalignment='left', bbox=props)

        print(f"Order Coefficient (radial + angular): {oc:.4f}")
        print(f"Order Coefficient (angular only): {oc_ang:.4f}")    

        if saveFigPath is not None:
            matplotlib.pyplot.savefig(saveFigPath)
            matplotlib.pyplot.close()
        else:
            
            matplotlib.pyplot.show()





   






def distributionDensity(F, step=50, lim=None, color=None, viewAnglesDeg=[25, 45], title=None, saveFigPath=None):
    """
    Creates the surface plot of the distribution density of the deviatoric fabric tensor F

    Parameters
    ----------
        F : 3x3 array of floats
            deviatoric fabric tensor. Usually obtained from spam.label.fabricTensor

        step : int, optional
            Number of points for the surface plot
            Default = 50

        lim : float, optional
            Limit for the axes of the plot
            Default = None

        color : colormap class, optional
            Colormap class from matplotlib module
            See 'https://matplotlib.org/3.1.0/tutorials/colors/colormaps.html' for options
            Example : matplotlib.pyplot.cm.viridis
            Default = matplotlib.pyplot.cm.Reds

        viewAnglesDeg : 2-component list, optional
            Set initial elevation and azimuth for this 3D plot

        title : str, optional
            Title for the graph
            Default = None

        saveFigPath : string, optional
            Path to save figure to.
            Default = None

    Returns
    -------
        None -- A matplotlib graph is created and shown

    Note
    ----
        see [Kanatani, 1984] for more information on the distribution density function for the deviatoric fabric tensor

    """
    # Create array of angles
    theta, phi = numpy.linspace(0, 2 * numpy.pi, step), numpy.linspace(0, numpy.pi, step)
    # Create meshgrid
    THETA, PHI = numpy.meshgrid(theta, phi)
    # Create radius array
    R = numpy.zeros(THETA.shape)
    # Copmute the radius for each angle
    for r in range(0, step, 1):
        for s in range(0, step, 1):
            vect = numpy.array((numpy.cos(phi[r]), numpy.sin(phi[r]) * numpy.sin(theta[s]), numpy.cos(theta[s]) * numpy.sin(phi[r])))
            R[r, s] = (1 / (4 * numpy.pi)) * (1 + numpy.dot(numpy.dot(F, vect), vect))
    # Change to cartesian coordinates
    X = R * numpy.sin(PHI) * numpy.cos(THETA)
    Y = R * numpy.sin(PHI) * numpy.sin(THETA)
    Z = R * numpy.cos(PHI)
    # Create figure
    import matplotlib

    matplotlib.rcParams.update({"font.size": 10})
    fig = matplotlib.pyplot.figure()
    ax = fig.add_subplot(111, projection="3d")
    # Set limits
    if lim == None:
        lim = round(numpy.max(R), 2)
    ax.set_xlim3d(-lim, lim)
    ax.set_ylim3d(-lim, lim)
    ax.set_zlim3d(-lim, lim)

    ax.view_init(viewAnglesDeg[0], viewAnglesDeg[1])
    # Set ticks
    ax.set_xticks((-lim, 0, lim))
    ax.set_yticks((-lim, 0, lim))
    ax.set_zticks((-lim, 0, lim))
    ax.set_box_aspect((1, 1, 1))
    # set axis titles
    ax.set_xlabel("X axis")
    ax.set_ylabel("Y axis")
    ax.set_zlabel("Z axis")
    # Title
    if title is not None:
        ax.set_title(str(title) + "\n")
    # Colormap
    if color == None:
        cmap = matplotlib.pyplot.get_cmap(matplotlib.pyplot.cm.Reds)
    else:
        cmap = matplotlib.pyplot.get_cmap(color)
    norm = mcolors.Normalize(vmin=0, vmax=Z.max())
    # Plot
    ax.plot_surface(
        X,
        Y,
        Z,
        rstride=1,
        cstride=1,
        # facecolors = cmap(norm(numpy.abs(Z))),
        # coloring by max extension
        facecolors=cmap((R - numpy.amin(R)) / numpy.amax(R - numpy.amin(R))),
        linewidth=0,
        antialiased=True,
        alpha=1,
    )

    matplotlib.pyplot.tight_layout()
    if saveFigPath is not None:
        matplotlib.pyplot.savefig(saveFigPath)
    else:
        matplotlib.pyplot.show()





def plotSphericalHistogram(orientations, subDiv=3, reflection=True, maxVal=None, verbose=True, color=None, viewAnglesDeg=[25, 45], title=None, saveFigPath=None):
    """
    Generates a spherical histogram for vectorial data, binning the data into regions defined by the faces of an icosphere (convex polyhedron made from triangles).

    The icosphere is built from starting from an icosahedron (polyhedron with 20 faces) and then making subdivision on each triangle.
    The number of faces is  20*(4**subDiv).

    Parameters
    ----------
        orientations : Nx3 numpy array
            Vectors to be plotted

        subDiv : integer, optional
            Number of times that the initial icosahedron is divided.
            Default: 3

        reflection : bool, optional
            If true, the histogram takes into account the reflection of the vectors
            Default = True.

        maxVal : int, optional
            Maximum colour-bar limits for bin view.
            Default = None (`i.e.`, auto-set)

        verbose : bool, optional
            Print the evolution of the plot
            Defautl = False

        color : colormap class, optional
            Colormap class from matplotlib module
            See 'https://matplotlib.org/3.1.0/tutorials/colors/colormaps.html' for options
            Example : matplotlib.pyplot.cm.viridis
            Default = matplotlib.pyplot.cm.viridis_r

        viewAnglesDeg : 2-component list, optional
            Set initial elevation and azimuth for this 3D plot

        title : str, optional
            Title for the graph
            Default = None

        saveFigPath : string, optional
            Path to save figure to, including the name and extension of the file.
            If it is not given, the plot will be shown but not saved.
            Default = None


    Returns
    -------
        None -- A matplotlib graph is created and shown

    """
    # Internal function for binning data into the icosphere faces

    def binIcosphere(data, icoVectors, verbose):
        # Create counts array
        counts = numpy.zeros(len(icoVectors))
        global computeAngle

        def computeAngle(i):
            # Get the orientation vector
            orientationVect = data[i]
            # Exchange Z and X position - for plotting
            orientationVect = [orientationVect[2], orientationVect[1], orientationVect[0]]
            # Create the result array
            angle = []
            for i in range(len(icoVectors)):
                # Compute the angle between them
                angle.append(numpy.arccos(numpy.clip(numpy.dot(orientationVect, icoVectors[i]), -1, 1)))
            # Get the index
            minIndex = numpy.argmin(angle)
            return minIndex

        # Create progressbar
        finishedOrientations = 0
        # Run multiprocessing
        with multiprocessing.Pool(processes=nProcessesDefault) as pool:
            for returns in pool.imap_unordered(computeAngle, range(len(data))):
                # Update the progressbar
                finishedOrientations += 1
                # Get the results
                index = returns
                # Add the count
                counts[index] += 1

        return counts

    # Get number of points
    numberOfPoints = orientations.shape[0]
    # Check that they are 3D vectors
    if orientations.shape[1] != 3:
        print("\nspam.helpers.orientationPlotter.plotSphericalHistogram: The input vectors are not 3D")
        return
    # from http://stackoverflow.com/questions/2850743/numpy-how-to-quickly-normalize-many-vectors
    norms = numpy.apply_along_axis(numpy.linalg.norm, 1, orientations)
    orientations = orientations / norms.reshape(-1, 1)
    # Check if we can reflect the vectors
    if reflection:
        orientations = numpy.vstack([orientations, -1 * orientations])
    # Create the icosphere
    if verbose:
        print("\nspam.helpers.orientationPlotter.plotSphericalHistogram: Creating the icosphere")
    icoVerts, icoFaces, icoVectors = orientationpy.generateIcosphere(subDiv)
    # Bin the data
    if verbose:
        print("\nspam.helpers.orientationPlotter.plotSphericalHistogram: Binning the data")
    counts = binIcosphere(orientations, icoVectors, verbose=verbose)
    # Now we are ready to plot
    if verbose:
        print("\nspam.helpers.orientationPlotter.plotSphericalHistogram: Plotting")

    # Create the figure
    fig = matplotlib.pyplot.figure()
    ax = fig.add_subplot(projection="3d")
    # ax = fig.gca(projection="3d")

    if color is None:
        cmap = matplotlib.pyplot.cm.viridis_r
    else:
        cmap = color
    norm = matplotlib.pyplot.Normalize(vmin=0, vmax=1)
    if maxVal is None:
        maxVal = numpy.max(counts)

    # Don't do it like this, make empty arrays!
    # points = []
    # connectivityMatrix = []

    # Loop through each of the faces
    for i in range(len(icoFaces)):
        # Get the corresponding radius
        radii = counts[i] / maxVal
        if radii != 0:
            # Get the face
            face = icoFaces[i]
            # Get the vertices
            P1 = numpy.asarray(icoVerts[face[0]])
            P2 = numpy.asarray(icoVerts[face[1]])
            P3 = numpy.asarray(icoVerts[face[2]])
            # Extend the vertices as needed by the radius
            P1 = radii * P1 / numpy.linalg.norm(P1)
            P2 = radii * P2 / numpy.linalg.norm(P2)
            P3 = radii * P3 / numpy.linalg.norm(P3)
            # Combine the vertices
            vertices = numpy.asarray([numpy.array([0, 0, 0]), P1, P2, P3])

            # for vertex in vertices:
            # points.append(vertex)
            # connectivityMatrix.append([len(points)-1, len(points)-2, len(points)-3, len(points)-4])

            # Add the points to the scatter3D
            ax.scatter3D(vertices[:, 0], vertices[:, 1], vertices[:, 2], s=0)
            # Create each face
            face1 = numpy.array([vertices[0], vertices[1], vertices[2]])
            face2 = numpy.array([vertices[0], vertices[1], vertices[3]])
            face3 = numpy.array([vertices[0], vertices[3], vertices[2]])
            face4 = numpy.array([vertices[3], vertices[1], vertices[2]])

            # Plot each face!
            ax.add_collection3d(Poly3DCollection([face1, face2, face3, face4], facecolors=cmap(norm(radii)), linewidths=0.5, edgecolors="k"))

    # Extra parameters for the axis
    ax.set_box_aspect([1, 1, 1])
    matplotlib.pyplot.xlim(-1.1, 1.1)
    matplotlib.pyplot.ylim(-1.1, 1.1)
    ax.set_zlim(-1.1, 1.1)
    ax.view_init(viewAnglesDeg[0], viewAnglesDeg[1])
    # Set the colorbar
    norm = matplotlib.colors.Normalize(vmin=0, vmax=maxVal)
    sm = matplotlib.pyplot.cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])
    matplotlib.pyplot.colorbar(sm, label="Number of vectors in bin", ax=ax)

    hideAxes = False
    if hideAxes:
        ax.xaxis.set_ticklabels([])
        ax.yaxis.set_ticklabels([])
        ax.zaxis.set_ticklabels([])

    else:
        ax.set_xlabel("X axis")
        ax.set_ylabel("Y axis")
        ax.set_zlabel("Z axis")
        ax.xaxis.set_ticks([-1, 0, 1])
        ax.yaxis.set_ticks([-1, 0, 1])
        ax.zaxis.set_ticks([-1, 0, 1])
        # ax.xaxis.set_ticklabels([-1, 0, 1])
        # ax.yaxis.set_ticklabels([-1, 0, 1])
        # ax.zaxis.set_ticklabels([-1, 0, 1])

    # Remove the ticks labels and lines
    ax = matplotlib.pyplot.gca()
    # for line in ax.xaxis.get_ticklines():
    # line.set_visible(False)
    # for line in ax.yaxis.get_ticklines():
    # line.set_visible(False)
    # for line in ax.zaxis.get_ticklines():
    # line.set_visible(False)
    # Title
    if title is not None:
        ax.set_title(str(title) + "\n")
    matplotlib.pyplot.tight_layout()
    if saveFigPath is not None:
        matplotlib.pyplot.savefig(saveFigPath)
    else:
        matplotlib.pyplot.show()

In [ ]:
def execute_tests(img_3d):

    import numpy as np
    import matplotlib.pyplot as plt
    from matplotlib import cm


    
    plt.imshow(img_3d[:, :, img_3d.shape[2] // 2], cmap="Greys_r")
    plt.title("Imagen de una ventana de la imagen 3D")
    plt.show()

    print("Computing structure tensor...", end="")
    structureTensor = computeGradientStructureTensor(img_3d, sigma=1, mode="gaussian")
    print("done.")


    intensity = computeIntensity(structureTensor)
    directionality = computeStructureDirectionality(structureTensor)
    structureType = computeStructureType(structureTensor)

    # 2. Definir umbrales (ajustar según necesidades)
    dark_threshold = 0.04 * np.max(intensity)  # 5% de la intensidad máxima
    anisotropy_threshold = 0.2  # umbral de coherencia para anisotropía
    # Máscaras booleanas para clasificación
    mask_dark = intensity < dark_threshold


    plt.figure(figsize=(10, 4))

    # The intentsity represents how strong the orientation signal is

    plt.imshow((intensity / intensity.max())[:, :, img_3d.shape[2] // 2], vmin=0, vmax=1)
    plt.colorbar()
    plt.title("Intensity Normalised")
    plt.show()

    # The directionality measures how strongly aligned the image is locally
    # directionality[numpy.isnan(directionality)] = 0
    directionality[img_3d== 0] = 0

    plt.figure(figsize=(10, 4))
    plt.imshow(directionality[:, :, img_3d.shape[2] // 2], norm=matplotlib.colors.LogNorm())
    plt.title("Directionality")
    plt.colorbar()
    plt.tight_layout()
    plt.show()
    print("Min:", np.min(structureTensor), "Max:", np.max(structureTensor))
    print("Shape:", structureTensor.shape)


    structureTensor += 1e-6  # Pequeña cantidad para evitar divisiones por cero
    print("Min:", np.min(structureTensor), "Max:", np.max(structureTensor))
    print("Computing theta and phi...", end="")
    orientations = computeOrientation(structureTensor, mode="fibre")
    print("done.")
    fig, ax = plt.subplots()
    imDisplayHSV = np.zeros((img_3d.shape[0], img_3d.shape[1], img_3d.shape[2], 3), dtype="f4")

    # Hue is the orientation (nice circular mapping)
    imDisplayHSV[:, :, :, 0] = orientations["phi"] / 360
    # Saturation is verticality, so white = up (z)
    imDisplayHSV[:, :, :, 1] = np.sin(np.deg2rad(orientations["theta"]))
    # Value is original image
    imDisplayHSV[:, :, :, 2] = img_3d / img_3d.max()

    fig.suptitle("Composition of image with 3D orientation")
    ax.set_title("H = Azimuth, S = Polar Angle, V = image")
    ax.imshow(matplotlib.colors.hsv_to_rgb(imDisplayHSV)[:, :, img_3d.shape[2] // 2])

    cmap = matplotlib.cm.hsv
    norm = matplotlib.colors.Normalize(vmin=0, vmax=360)
    fig.colorbar(
        matplotlib.cm.ScalarMappable(norm=norm, cmap=cmap),
        ax=ax,
        orientation="vertical",
        label="azimuth angle (degrees in xy-plane from +x)",
    )

    plt.show()


    boxSizePixels = 5
    structureTensorBoxes = computeGradientStructureTensorBoxes(
        img_3d,
        [boxSizePixels, boxSizePixels, boxSizePixels],
    )

    # 1. Calcular intensidad promedio por caja (traza del tensor de estructura)
    intensity_boxes = computeIntensity(structureTensorBoxes)

    # 2. Definir umbrales (ajustar según necesidades)
    dark_threshold = 0.04 * np.max(intensity_boxes)  # 5% de la intensidad máxima
    anisotropy_threshold = 0.2  # umbral de coherencia para anisotropía

    # 3. Clasificar regiones
    coherence_map = computeCoherenceMap(structureTensorBoxes)

    # Máscaras booleanas para clasificación
    mask_dark = intensity_boxes < dark_threshold
    mask_isotropic = (~mask_dark) & (coherence_map < anisotropy_threshold)
    mask_anisotropic = (~mask_dark) & (coherence_map >= anisotropy_threshold)

    # 5. Generar estadísticas
    total_boxes = np.prod(intensity_boxes.shape)
    stats = {
        "dark": np.sum(mask_dark),
        "isotropic": np.sum(mask_isotropic),
        "anisotropic": np.sum(mask_anisotropic),
        "total": total_boxes
    }

    print("\nESTADÍSTICAS DE REGIONES:")
    print(f"Oscuras: {stats['dark']} ({stats['dark']/stats['total']*100:.1f}%)")
    print(f"Isotrópicas: {stats['isotropic']} ({stats['isotropic']/stats['total']*100:.1f}%)")
    print(f"Anisotrópicas: {stats['anisotropic']} ({stats['anisotropic']/stats['total']*100:.1f}%)")

    # 4. Visualización solo de regiones anisotrópicas
    orientationsBoxes = computeOrientation(
        structureTensorBoxes,
        mode="fiber",
    )

    boxVectorsZYX = anglesToVectors(orientationsBoxes)
    mask_not_dark = intensity_boxes >= dark_threshold


    # Centros de caja (solo para regiones anisotrópicas)
    boxCentresX, boxCentresY, boxCentresZ = np.mgrid[
        boxSizePixels//2 : mask_not_dark.shape[0]*boxSizePixels : boxSizePixels,
        boxSizePixels//2 : mask_not_dark.shape[1]*boxSizePixels : boxSizePixels,
        boxSizePixels//2 : mask_not_dark.shape[2]*boxSizePixels : boxSizePixels,
    ]

    # Filtrar solo regiones anisotrópicas
    X = boxCentresX[mask_not_dark]
    Y = boxCentresY[mask_not_dark]
    Z = boxCentresZ[mask_not_dark]
    U = boxVectorsZYX[2][mask_not_dark]
    V = boxVectorsZYX[1][mask_not_dark]
    W = boxVectorsZYX[0][mask_not_dark]

    # Asignar color por Z (fotograma)
    norm = plt.Normalize(Z.min(), Z.max())
    colors = cm.viridis(norm(Z))

    # Reducción de flechas si son muchas
    max_arrows = 800
    if len(X) > max_arrows:
        idx = np.random.choice(len(X), max_arrows, replace=False)
        X, Y, Z = X[idx], Y[idx], Z[idx]
        U, V, W = U[idx], V[idx], W[idx]
        colors = colors[idx]

    # Visualización 3D
    fig = plt.figure(figsize=(12, 10))
    ax = fig.add_subplot(111, projection="3d")

    ax.quiver(
        X, Y, Z, U, V, W,
        length=boxSizePixels*0.8,
        normalize=True,
        linewidth=1.2,
        color=colors,
        alpha=0.85,
        arrow_length_ratio=0.35
    )

    ax.set_title("Orientación 3D de Fibras (Regiones Anisotrópicas)", fontsize=15)
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.view_init(elev=30, azim=45)
    ax.invert_zaxis()

    # Barra de color para Z
    mappable = cm.ScalarMappable(cmap='viridis', norm=norm)
    mappable.set_array(Z)
    cbar = fig.colorbar(mappable, ax=ax, shrink=0.7, pad=0.1)
    cbar.set_label('Fotograma (Z)', fontsize=12)

    plt.tight_layout()
    plt.show()

    # Visualización de clasificación de regiones (opcional)
    fig2, ax2 = plt.subplots(1, 1, figsize=(8, 6))
    categories = ['Oscuras', 'Isotrópicas', 'Anisotrópicas']
    counts = [stats['dark'], stats['isotropic'], stats['anisotropic']]

    ax2.bar(categories, counts, color=['gray', 'blue', 'red'])
    ax2.set_title("Distribución de Regiones")
    ax2.set_ylabel("Número de Cajas")
    plt.show()

    # Calcular métricas cuantitativas
    metrics = compute_orientation_metrics(structureTensorBoxes)

    # Imprimir resultados
    print("Coherencia media por capa:")
    for z, coh in enumerate(metrics['mean_coherence_per_layer']):
        print(f"  Capa {z}: {coh:.4f}")

    metrics = compute_orientation_metrics(structureTensorBoxes)
    print("\nMÉTRICAS DE ORIENTACIÓN:")
    print(f"Coherencia media global: {metrics['global_mean_coherence']:.4f}")



    # --- Calcular y mostrar el histograma de orientaciones y el coeficiente de orden (DoI) ---
    # Vector de orientaciones solo en regiones NO oscuras


    # --- Máscara para regiones NO oscuras ---
    mask_not_dark = intensity_boxes >= dark_threshold
    # --- Filtrar solo regiones NO oscuras ---
    X = boxCentresX[mask_not_dark]
    Y = boxCentresY[mask_not_dark]
    Z = boxCentresZ[mask_not_dark]
    U = boxVectorsZYX[2][mask_not_dark]
    V = boxVectorsZYX[1][mask_not_dark]
    W = boxVectorsZYX[0][mask_not_dark]


    orientationVectors_not_dark = np.stack([W, V, U], axis=1)  # ZYX

    # Histograma y DoI solo en regiones NO oscuras
    plotOrientations3d(orientationVectors_not_dark, pointMarkerSize=1)




    

In [ ]:
def stack_images_to_3d(folder_path):
    """Apila imágenes 2D en un volumen 3D."""
    image_files = sorted([f for f in os.listdir(folder_path) if f.endswith(('.png', '.jpg', '.tif'))])
    if not image_files:
        raise ValueError("No se encontraron imágenes en la carpeta")
    
    images = [load_image(os.path.join(folder_path, f)) for f in image_files]
    volume = np.stack(images, axis=-1)  # Apilar en Z
    return volume

img_3d = stack_images_to_3d(folder_path_untreated)
execute_tests(img_3d)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm

import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Tamaño de la ventana cuadrada
window_size = 100  # píxeles

def load_image(image_path):
    """Carga una imagen en escala de grises."""
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        raise ValueError(f"No se pudo cargar la imagen: {image_path}")
    return img

def stack_images_to_3d(folder_path):
    """Apila imágenes 2D en un volumen 3D."""
    image_files = sorted([f for f in os.listdir(folder_path) if f.endswith(('.png', '.jpg', '.tif'))])
    if not image_files:
        raise ValueError("No se encontraron imágenes en la carpeta")
    
    images = [load_image(os.path.join(folder_path, f)) for f in image_files]
    volume = np.stack(images, axis=-1)  # Apilar en Z
    return volume

# Cargar volumen completo
img_3d = stack_images_to_3d(folder_path_untreated)

# Medidas
h, w, z = img_3d.shape

# Coordenadas (Y, X) de las esquinas superiores izquierdas de las ventanas
window_coords = [
    (1400, 1000),
    (200, w - window_size - 400),
    (550, 1100),
    (h - window_size - 200, 1000)
]

# Visualizar ventanas sobre una imagen representativa (ej: la del medio)
middle_slice = img_3d[:, :, 0]

fig, ax = plt.subplots()
ax.imshow(middle_slice, cmap="gray")
ax.set_title("Ubicación de las ventanas en la imagen 3D (XY view)")

for idx, (y, x) in enumerate(window_coords):
    rect = patches.Rectangle(
        (x, y), window_size, window_size,
        linewidth=2, edgecolor='red', facecolor='none'
    )
    ax.add_patch(rect)
    ax.text(x, y - 5, f"Ventana {idx+1}", color='red', fontsize=8, weight='bold')

plt.tight_layout()
plt.show()

# Extraer las ventanas 3D y almacenarlas en variables
ventana1 = img_3d[window_coords[0][0]:window_coords[0][0]+window_size, window_coords[0][1]:window_coords[0][1]+window_size, :]
ventana2 = img_3d[window_coords[1][0]:window_coords[1][0]+window_size, window_coords[1][1]:window_coords[1][1]+window_size, :]
ventana3 = img_3d[window_coords[2][0]:window_coords[2][0]+window_size, window_coords[2][1]:window_coords[2][1]+window_size, :]
ventana4 = img_3d[window_coords[3][0]:window_coords[3][0]+window_size, window_coords[3][1]:window_coords[3][1]+window_size]

# Mostrar información sobre las ventanas extraídas
print(f"Ventana 1: {ventana1.shape}")
print(f"Ventana 2: {ventana2.shape}")
print(f"Ventana 3: {ventana3.shape}")
print(f"Ventana 4: {ventana4.shape}")



boxSizePixels = 5
structureTensorBoxes = computeGradientStructureTensorBoxes(
    ventana1,
    [boxSizePixels, boxSizePixels, boxSizePixels],
)

# 1. Calcular intensidad promedio por caja (traza del tensor de estructura)
intensity_boxes = computeIntensity(structureTensorBoxes)

# 2. Definir umbrales (ajustar según necesidades)
dark_threshold = 0.04 * np.max(intensity_boxes)  # 5% de la intensidad máxima
anisotropy_threshold = 0.2  # umbral de coherencia para anisotropía

# 3. Clasificar regiones
coherence_map = computeCoherenceMap(structureTensorBoxes)

# Máscaras booleanas para clasificación
mask_dark = intensity_boxes < dark_threshold
mask_isotropic = (~mask_dark) & (coherence_map < anisotropy_threshold)
mask_anisotropic = (~mask_dark) & (coherence_map >= anisotropy_threshold)

# 5. Generar estadísticas
total_boxes = np.prod(intensity_boxes.shape)
stats = {
    "dark": np.sum(mask_dark),
    "isotropic": np.sum(mask_isotropic),
    "anisotropic": np.sum(mask_anisotropic),
    "total": total_boxes
}

print("\nESTADÍSTICAS DE REGIONES:")
print(f"Oscuras: {stats['dark']} ({stats['dark']/stats['total']*100:.1f}%)")
print(f"Isotrópicas: {stats['isotropic']} ({stats['isotropic']/stats['total']*100:.1f}%)")
print(f"Anisotrópicas: {stats['anisotropic']} ({stats['anisotropic']/stats['total']*100:.1f}%)")

# 4. Visualización solo de regiones anisotrópicas
orientationsBoxes = computeOrientation(
    structureTensorBoxes,
    mode="fiber",
)

boxVectorsZYX = anglesToVectors(orientationsBoxes)

# Centros de caja (solo para regiones anisotrópicas)
boxCentresX, boxCentresY, boxCentresZ = np.mgrid[
    boxSizePixels//2 : mask_anisotropic.shape[0]*boxSizePixels : boxSizePixels,
    boxSizePixels//2 : mask_anisotropic.shape[1]*boxSizePixels : boxSizePixels,
    boxSizePixels//2 : mask_anisotropic.shape[2]*boxSizePixels : boxSizePixels,
]

# Filtrar solo regiones anisotrópicas
X = boxCentresX[mask_anisotropic]
Y = boxCentresY[mask_anisotropic]
Z = boxCentresZ[mask_anisotropic]
U = boxVectorsZYX[2][mask_anisotropic]
V = boxVectorsZYX[1][mask_anisotropic]
W = boxVectorsZYX[0][mask_anisotropic]

# Asignar color por Z (fotograma)
norm = plt.Normalize(Z.min(), Z.max())
colors = cm.viridis(norm(Z))

# Reducción de flechas si son muchas
max_arrows = 800
if len(X) > max_arrows:
    idx = np.random.choice(len(X), max_arrows, replace=False)
    X, Y, Z = X[idx], Y[idx], Z[idx]
    U, V, W = U[idx], V[idx], W[idx]
    colors = colors[idx]

# Visualización 3D
fig = plt.figure(figsize=(12, 10))
ax = fig.add_subplot(111, projection="3d")

ax.quiver(
    X, Y, Z, U, V, W,
    length=boxSizePixels*0.8,
    normalize=True,
    linewidth=1.2,
    color=colors,
    alpha=0.85,
    arrow_length_ratio=0.35
)

ax.set_title("Orientación 3D de Fibras (Regiones Anisotrópicas)", fontsize=15)
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.view_init(elev=30, azim=45)
ax.invert_zaxis()

# Barra de color para Z
mappable = cm.ScalarMappable(cmap='viridis', norm=norm)
mappable.set_array(Z)
cbar = fig.colorbar(mappable, ax=ax, shrink=0.7, pad=0.1)
cbar.set_label('Fotograma (Z)', fontsize=12)

plt.tight_layout()
plt.show()

# Visualización de clasificación de regiones (opcional)
fig2, ax2 = plt.subplots(1, 1, figsize=(8, 6))
categories = ['Oscuras', 'Isotrópicas', 'Anisotrópicas']
counts = [stats['dark'], stats['isotropic'], stats['anisotropic']]

ax2.bar(categories, counts, color=['gray', 'blue', 'red'])
ax2.set_title("Distribución de Regiones")
ax2.set_ylabel("Número de Cajas")
plt.show()

# Calcular métricas cuantitativas
metrics = compute_orientation_metrics(structureTensorBoxes)

# Imprimir resultados
print("Coherencia media por capa:")
for z, coh in enumerate(metrics['mean_coherence_per_layer']):
    print(f"  Capa {z}: {coh:.4f}")

metrics = compute_orientation_metrics(structureTensorBoxes)
print("\nMÉTRICAS DE ORIENTACIÓN:")
print(f"Coherencia media global: {metrics['global_mean_coherence']:.4f}")

In [ ]:
execute_tests(ventana1)

In [ ]:
def partition_3d_stack(image_3d, num_partitions):
    """
    Divide un volumen 3D en un número especificado de particiones iguales (o casi iguales).
    
    :param image_3d: numpy array de forma (H, W, D)
    :param num_partitions: int, número de particiones a realizar
    :return: lista de arrays 3D, cada uno una partición del volumen original
    """
    if image_3d.ndim != 3:
        raise ValueError("La imagen debe ser un volumen 3D (H, W, D)")
    
    h, w, d = image_3d.shape
    if num_partitions > d:
        raise ValueError("El número de particiones no puede ser mayor que el número de cortes Z")
    
    # Calculamos los índices de partición
    partition_size = d // num_partitions
    remainder = d % num_partitions

    partitions = []
    start = 0
    for i in range(num_partitions):
        extra = 1 if i < remainder else 0
        end = start + partition_size + extra
        partitions.append(image_3d[:, :, start:end])
        start = end

    return partitions






In [ ]:

partitions=partition_3d_stack(img_3d, num_partitions=4)
for i, subvol in enumerate(partitions):
    print("Computing structure tensor...", end="")
    structureTensor = computeGradientStructureTensor(subvol, sigma=1, mode="gaussian")
    print("done.")
    intensity = computeIntensity(structureTensor)
    directionality = computeStructureDirectionality(structureTensor)
    structureType = computeStructureType(structureTensor)
    structureTensor += 1e-6  # Pequeña cantidad para evitar divisiones por cero
    print("Computing theta and phi...", end="")
    orientations = computeOrientation(structureTensor, mode="fibre")
    print("done.")
    boxSizePixels = 5
    structureTensorBoxes = computeGradientStructureTensorBoxes(
        subvol,
        [boxSizePixels, boxSizePixels, boxSizePixels],
        )

    # --- Máscara para regiones NO oscuras ---
    mask_not_dark = intensity_boxes >= dark_threshold
    # --- Filtrar solo regiones NO oscuras ---
    X = boxCentresX[mask_not_dark]
    Y = boxCentresY[mask_not_dark]
    Z = boxCentresZ[mask_not_dark]
    U = boxVectorsZYX[2][mask_not_dark]
    V = boxVectorsZYX[1][mask_not_dark]
    W = boxVectorsZYX[0][mask_not_dark]


    orientationVectors_not_dark = np.stack([W, V, U], axis=1)  # ZYX


    orientationVectors = boxVectorsZYX.reshape(3, -1).T

    print(f"Resultados partición {i+1}")
    plotOrientations3d(orientationVectors, pointMarkerSize=1)



In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from sklearn.cluster import KMeans
from scipy.fftpack import fftn, fftshift, ifftn
from scipy.ndimage import rotate, binary_opening
from skimage.filters.rank import entropy
from skimage.morphology import disk


#--------------Esto es una función que voy a escribir yo ahora para que me haga lo de el volumnen 3D y la distribución de regiones, la cual tenía pero no en una función-------------------


import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm

import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches


def fibers_in_a_3D_volume(img_3d, boxSizePixels = 5):
    structureTensorBoxes = computeGradientStructureTensorBoxes(
        img_3d,
        [boxSizePixels, boxSizePixels, boxSizePixels],
    )

    # 1. Calcular intensidad promedio por caja (traza del tensor de estructura)
    intensity_boxes = computeIntensity(structureTensorBoxes)

    # 2. Definir umbrales (ajustar según necesidades)
    dark_threshold = 0.04 * np.max(intensity_boxes)  # 5% de la intensidad máxima
    anisotropy_threshold = 0.2  # umbral de coherencia para anisotropía

    # 3. Clasificar regiones
    coherence_map = computeCoherenceMap(structureTensorBoxes)

    # Máscaras booleanas para clasificación
    mask_dark = intensity_boxes < dark_threshold
    mask_isotropic = (~mask_dark) & (coherence_map < anisotropy_threshold)
    mask_anisotropic = (~mask_dark) & (coherence_map >= anisotropy_threshold)

    # 5. Generar estadísticas
    total_boxes = np.prod(intensity_boxes.shape)
    stats = {
        "dark": np.sum(mask_dark),
        "isotropic": np.sum(mask_isotropic),
        "anisotropic": np.sum(mask_anisotropic),
        "total": total_boxes
    }

    print("\nESTADÍSTICAS DE REGIONES:")
    print(f"Oscuras: {stats['dark']} ({stats['dark']/stats['total']*100:.1f}%)")
    print(f"Isotrópicas: {stats['isotropic']} ({stats['isotropic']/stats['total']*100:.1f}%)")
    print(f"Anisotrópicas: {stats['anisotropic']} ({stats['anisotropic']/stats['total']*100:.1f}%)")

    # 4. Visualización solo de regiones anisotrópicas
    orientationsBoxes = computeOrientation(
        structureTensorBoxes,
        mode="fiber",
    )

    boxVectorsZYX = anglesToVectors(orientationsBoxes)

    # Centros de caja (solo para regiones anisotrópicas)
    boxCentresX, boxCentresY, boxCentresZ = np.mgrid[
        boxSizePixels//2 : mask_anisotropic.shape[0]*boxSizePixels : boxSizePixels,
        boxSizePixels//2 : mask_anisotropic.shape[1]*boxSizePixels : boxSizePixels,
        boxSizePixels//2 : mask_anisotropic.shape[2]*boxSizePixels : boxSizePixels,
    ]

    # Filtrar solo regiones anisotrópicas
    X = boxCentresX[mask_anisotropic]
    Y = boxCentresY[mask_anisotropic]
    Z = boxCentresZ[mask_anisotropic]
    U = boxVectorsZYX[2][mask_anisotropic]
    V = boxVectorsZYX[1][mask_anisotropic]
    W = boxVectorsZYX[0][mask_anisotropic]

    # Asignar color por Z (fotograma)
    norm = plt.Normalize(Z.min(), Z.max())
    colors = cm.viridis(norm(Z))

    # Reducción de flechas si son muchas
    max_arrows = 800
    if len(X) > max_arrows:
        idx = np.random.choice(len(X), max_arrows, replace=False)
        X, Y, Z = X[idx], Y[idx], Z[idx]
        U, V, W = U[idx], V[idx], W[idx]
        colors = colors[idx]

    # Visualización 3D
    fig = plt.figure(figsize=(12, 10))
    ax = fig.add_subplot(111, projection="3d")

    ax.quiver(
        X, Y, Z, U, V, W,
        length=boxSizePixels*0.8,
        normalize=True,
        linewidth=1.2,
        color=colors,
        alpha=0.85,
        arrow_length_ratio=0.35
    )

    ax.set_title("Orientación 3D de Fibras (Regiones Anisotrópicas)", fontsize=15)
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.view_init(elev=30, azim=45)
    ax.invert_zaxis()

    # Barra de color para Z
    mappable = cm.ScalarMappable(cmap='viridis', norm=norm)
    mappable.set_array(Z)
    cbar = fig.colorbar(mappable, ax=ax, shrink=0.7, pad=0.1)
    cbar.set_label('Fotograma (Z)', fontsize=12)

    plt.tight_layout()
    plt.show()

    # Visualización de clasificación de regiones (opcional)
    fig2, ax2 = plt.subplots(1, 1, figsize=(8, 6))
    categories = ['Oscuras', 'Isotrópicas', 'Anisotrópicas']
    counts = [stats['dark'], stats['isotropic'], stats['anisotropic']]

    ax2.bar(categories, counts, color=['gray', 'blue', 'red'])
    ax2.set_title("Distribución de Regiones")
    ax2.set_ylabel("Número de Cajas")
    plt.show()

    # Calcular métricas cuantitativas
    metrics = compute_orientation_metrics(structureTensorBoxes)

    # Imprimir resultados
    print("Coherencia media por capa:")
    for z, coh in enumerate(metrics['mean_coherence_per_layer']):
        print(f"  Capa {z}: {coh:.4f}")

    metrics = compute_orientation_metrics(structureTensorBoxes)
    print("\nMÉTRICAS DE ORIENTACIÓN:")
    print(f"Coherencia media global: {metrics['global_mean_coherence']:.4f}")



# Función Hough modificada para solo retornar intersecciones sin mostrar gráficos
def hough_transform_intensity_filtered(reconstructed_img_lines, intensity_percentile=90, entropy_filter=True, show_plots=False):
    # Suavizado y normalización
    img_blur = cv2.GaussianBlur(reconstructed_img_lines, (5, 5), 0)
    img_norm = np.uint8(255 * img_blur / np.max(img_blur))

    # Filtrado por intensidad (percentil alto)
    threshold = np.percentile(img_norm, intensity_percentile)
    mask_intensity = img_norm > threshold

    # Filtrado por entropía local
    if entropy_filter:
        ent = entropy(img_norm, disk(5))
        ent_norm = (ent - ent.min()) / (ent.max() - ent.min())
        mask_entropy = ent_norm > 0.6
        mask = mask_intensity & mask_entropy
    else:
        mask = mask_intensity

    # Convertir la máscara a uint8 para Hough
    mask_uint8 = np.uint8(mask) * 255

    # Transformada de Hough sobre la máscara
    lines = cv2.HoughLines(mask_uint8, rho=1, theta=np.pi/180, threshold=30)
    intersections = []

    if lines is not None:
        # Calcular intersecciones entre pares de líneas
        for i in range(min(len(lines), 100)):
            for j in range(i+1, min(len(lines), 100)):
                rho1, theta1 = lines[i][0]
                rho2, theta2 = lines[j][0]
                if abs(theta1 - theta2) > np.deg2rad(10):
                    A = np.array([
                        [np.cos(theta1), np.sin(theta1)],
                        [np.cos(theta2), np.sin(theta2)]
                    ])
                    b = np.array([[rho1], [rho2]])
                    try:
                        x0, y0 = np.linalg.solve(A, b)
                        x0, y0 = int(x0), int(y0)
                        if 0 <= x0 < mask_uint8.shape[1] and 0 <= y0 < mask_uint8.shape[0]:
                            intersections.append((x0, y0))
                    except np.linalg.LinAlgError:
                        continue

    # Visualización opcional
    if show_plots:
        hough_image = cv2.cvtColor(img_norm, cv2.COLOR_GRAY2BGR)
        
        # Dibujar líneas
        for i in range(min(len(lines), 100)):
            rho, theta = lines[i][0]
            a = np.cos(theta)
            b = np.sin(theta)
            x0 = a * rho
            y0 = b * rho
            pt1 = (int(x0 + 1000*(-b)), int(y0 + 1000*(a)))
            pt2 = (int(x0 - 1000*(-b)), int(y0 - 1000*(a)))
            cv2.line(hough_image, pt1, pt2, (0, 0, 255), 1)
        
        # Dibujar intersecciones
        for (x, y) in intersections:
            cv2.circle(hough_image, (x, y), 3, (0, 255, 0), -1)

        plt.figure(figsize=(15, 5))
        plt.subplot(1, 3, 1)
        plt.imshow(img_norm, cmap='gray')
        plt.title("Reconstruida normalizada")
        plt.axis('off')

        plt.subplot(1, 3, 2)
        plt.imshow(mask_uint8, cmap='gray')
        plt.title("Máscara intensidad + entropía")
        plt.axis('off')

        plt.subplot(1, 3, 3)
        plt.imshow(hough_image[..., ::-1])
        plt.title("Hough sobre máscara")
        plt.axis('off')
        plt.tight_layout()
        plt.show()
    
    return intersections

# Función para cargar una imagen en escala de grises
def load_image(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        raise ValueError(f"No se pudo cargar la imagen: {image_path}")
    return img

# Función para calcular la Transformada de Fourier 2D
def compute_fft_2d(img):
    f_transform = fftn(img)  
    f_transform_shifted = fftshift(f_transform)  
    magnitude_spectrum = np.log1p(np.abs(f_transform_shifted))  
    return f_transform_shifted, magnitude_spectrum

# Filtro por porcentaje de energía
def filter_by_energy(f_transform, energy_percent=20):
    magnitude = np.abs(f_transform)
    total_energy = np.sum(magnitude)
    sorted_magnitude = np.sort(magnitude.ravel())[::-1]
    cumulative_energy = np.cumsum(sorted_magnitude)
    threshold_index = np.searchsorted(cumulative_energy, total_energy * (energy_percent / 100))
    threshold_value = sorted_magnitude[threshold_index]
    mask = magnitude >= threshold_value
    filtered_transform = f_transform * mask
    return filtered_transform, mask

# Filtrar puntos dispersos en la máscara
def clean_mask(mask):
    cleaned_mask = binary_opening(mask, structure=np.ones((3, 3)))
    return cleaned_mask

# Calcular la dirección principal del espectro filtrado
def calculate_main_directions(mask, magnitude=None, num_directions=3):
    h, w = mask.shape
    cy, cx = h // 2, w // 2
    mask = clean_mask(mask)
    y, x = np.nonzero(mask)
    
    if len(x) == 0 or len(y) == 0:
        return [0]
    
    y = y - cy
    x = x - cx
    range_x = np.max(x) - np.min(x)
    range_y = np.max(y) - np.min(y)
    aspect_ratio = max(range_x, 1) / max(range_y, 1)

    if aspect_ratio > 1.7 or aspect_ratio < 0.6:
        A = np.vstack([x, np.ones(len(x))]).T
        m, _ = np.linalg.lstsq(A, y, rcond=None)[0]
        main_direction = np.arctan(m)
        return [np.degrees(main_direction)]
    else:
        if magnitude is not None:
            weights = np.abs(magnitude)[mask]
            weights /= np.sum(weights)
            cov_matrix = np.cov(x, y, aweights=weights)
        else:
            cov_matrix = np.cov(x, y)
        
        eigenvalues, eigenvectors = np.linalg.eig(cov_matrix)
        sorted_indices = np.argsort(eigenvalues)[::-1]
        eigenvectors = eigenvectors[:, sorted_indices]
        
        directions = []
        for i in range(min(num_directions, len(eigenvalues))):
            angle = np.arctan2(eigenvectors[1, i], eigenvectors[0, i])
            directions.append(np.degrees(angle))
        
        while len(directions) < num_directions:
            directions.append((360 / num_directions) * len(directions))
        
        return directions

# Filtro de dos líneas delgadas rotadas
def filter_two_lines_rotated(f_transform, angle):
    h, w = f_transform.shape
    cy, cx = h // 2, w // 2
    mask = np.zeros((h, w), dtype=np.uint8)
    mask[cy - 1:cy + 2, :] = 1
    mask[:, cx - 1:cx + 2] = 1
    rotated_mask = rotate(mask, angle, reshape=False, order=1)
    filtered_transform = f_transform * rotated_mask
    return filtered_transform, rotated_mask

# Apilar imágenes en volumen 3D
def stack_images_to_3d(folder_path):
    image_files = sorted([f for f in os.listdir(folder_path) if f.endswith(('.png', '.jpg', '.tif'))])
    if not image_files:
        raise ValueError("No se encontraron imágenes en la carpeta")
    
    images = [load_image(os.path.join(folder_path, f)) for f in image_files]
    volume = np.stack(images, axis=-1)
    return volume

# Obtener intersecciones Hough para una imagen
def get_hough_intersections(img, show_plots=False):
    f_transform_shifted, _ = compute_fft_2d(img)
    filtered_transform_energy, mask_energy = filter_by_energy(f_transform_shifted, 20)
    main_directions = calculate_main_directions(mask_energy, np.abs(f_transform_shifted), 3)
    
    combined_filtered_transform = np.zeros_like(f_transform_shifted)
    for direction in main_directions:
        filtered_transform_lines, _ = filter_two_lines_rotated(f_transform_shifted, direction)
        combined_filtered_transform += filtered_transform_lines
    
    reconstructed_img_lines = np.abs(ifftn(combined_filtered_transform))
    return hough_transform_intensity_filtered(reconstructed_img_lines, show_plots=show_plots)

# Función principal
def main(folder_path = "C:\\Users\\danwo\\Desktop\\ZS0001_TI_25XW_Rhodopsin_Au", window_size = 100 ):
    # 1. Cargar volumen 3D
    
    img_3d = stack_images_to_3d(folder_path)
    h, w, z = img_3d.shape
    
    # 2. Procesar TODOS los slices para obtener intersecciones
    all_intersections = []
    total_slices = z
    
    for i in range(total_slices):
        print(f"Procesando slice {i+1}/{total_slices}")
        slice_img = img_3d[:, :, i]
        intersections = get_hough_intersections(slice_img, show_plots=False)
        all_intersections.extend(intersections)

    # 3. Si no hay suficientes intersecciones, agregar predeterminadas
    if len(all_intersections) < 4:
        print("Advertencia: Pocas intersecciones detectadas. Agregando puntos predeterminados.")
        default_points = [
            (w // 4, h // 4),
            (3 * w // 4, h // 4),
            (w // 4, 3 * h // 4),
            (3 * w // 4, 3 * h // 4)
        ]
        all_intersections.extend(default_points)
    
    # 4. Agrupar intersecciones y seleccionar 4 principales
    kmeans = KMeans(n_clusters=4, random_state=0, n_init=10).fit(all_intersections)
    window_centers = kmeans.cluster_centers_.astype(int)
    
    # 5. Convertir centros a esquinas superiores izquierdas
    window_coords = []
    for center in window_centers:
        x, y = center
        x_top = max(0, min(x - window_size // 2, w - window_size))
        y_top = max(0, min(y - window_size // 2, h - window_size))
        window_coords.append((y_top, x_top))

    # 6. Visualizar ventanas en slice central
    middle_slice = img_3d[:, :, z // 2]
    fig, ax = plt.subplots(figsize=(12, 8))
    ax.imshow(middle_slice, cmap="gray")
    ax.set_title(f"Ventanas basadas en {len(all_intersections)} intersecciones de {z} slices")
    

    # Mostrar todas las intersecciones detectadas
    if all_intersections:
        intersections_arr = np.array(all_intersections)
        ax.scatter(intersections_arr[:, 0], intersections_arr[:, 1], 
                  s=20, c='cyan', marker='o', alpha=0.3, label="Intersecciones")
        ax.scatter(window_centers[:, 0], window_centers[:, 1], 
                  s=100, c='yellow', marker='X', edgecolors='red', label="Centroides")
        ax.legend()
    
    
    for idx, (y, x) in enumerate(window_coords):
        rect = patches.Rectangle(
            (x, y), window_size, window_size,
            linewidth=2, edgecolor='red', facecolor='none'
        )
        ax.add_patch(rect)
        ax.text(x, y - 10, f"Ventana {idx+1}", color='red', fontsize=10, weight='bold')
    



    plt.tight_layout()
    plt.show()
    
    # 7. Extraer volúmenes de ventanas
    ventanas = []
    for i, (y, x) in enumerate(window_coords):
        ventana = img_3d[y:y+window_size, x:x+window_size, :]
        ventanas.append(ventana)
        print(f"Ventana {i+1}: Pos=({y},{x}), Shape={ventana.shape}")
    
    return ventanas

if __name__ == "__main__":
    # Tamaño de la ventana cuadrada
    window_size = 100  # píxeles
    ventanas = main(folder_path_untreated, window_size)
    #Vamos a hacer para las 4 ventanas todo el análisis que tenemos
    for i, ventana in enumerate(ventanas):
        print("\n"+"\n" + "\033[1m" + f" ----------  Resultados análisis ventana {i+1} ------------ " + "\033[0m\n"+"\n")        
        fibers_in_a_3D_volume(ventana, boxSizePixels=5)
        execute_tests(ventana)


AQUÍ VOY A HACER LOS MISMOS RESULTADOS PERO PARA EL OJO TRATADO

In [ ]:
image= get_frame_from_folder(folder_path_treated, 0) 



import matplotlib
import matplotlib.colors
import matplotlib.pyplot as plt
import numpy as np
import orientationpy
import tifffile
import matplotlib.image as mpimg
import cv2

if image is None:
    raise FileNotFoundError(f"No se pudo cargar la imagen")

print(f"Dimensiones de la imagen: {image.shape}")

# Mostrar la imagen original
plt.figure(figsize=(10, 4))
plt.imshow(image, cmap="gray")
plt.title("Imagen original")
plt.show()


# Calcular gradientes con diferentes métodos
methods = ["finite_difference", "gaussian", "splines"]
fig, axes = plt.subplots(len(methods), 2, figsize=(10, 10))

for i, mode in enumerate(methods):
    gradients = orientationpy.computeGradient(image, mode=mode)
    Gy, Gx = gradients[-2:]  # Extraer solo Gy y Gx
    
    # Mostrar gradientes
    axes[i, 0].imshow(Gy, cmap="coolwarm", vmin=-64, vmax=64)
    axes[i, 0].set_title(f"{mode} - Gy")
    
    axes[i, 1].imshow(Gx, cmap="coolwarm", vmin=-64, vmax=64)
    axes[i, 1].set_title(f"{mode} - Gx")

plt.tight_layout()
plt.show()

# Calcular el tensor de estructura
structureTensor = orientationpy.computeStructureTensor([Gy, Gx], sigma=2)

# Calcular la dirección y orientación
orientations = orientationpy.computeOrientation(structureTensor)
directionality = np.linalg.norm(structureTensor, axis=0)

# Asegurar valores positivos para normalización logarítmica
directionality = np.where(~np.isfinite(directionality) | (directionality <= 0), 1e-10, directionality)

plt.figure(figsize=(10, 4))

# Mostrar intensidad normalizada

plt.imshow(directionality / directionality.max(), vmin=0, vmax=1)
plt.colorbar(shrink=0.7)
plt.title("Intensity Normalised")
plt.show()

# Mostrar dirección con normalización logarítmica
plt.figure(figsize=(10, 4))
plt.imshow(directionality, norm=matplotlib.colors.LogNorm(vmin=1e-10, vmax=directionality.max()))
plt.title("Directionality Normalised")
plt.colorbar(shrink=0.7)

plt.tight_layout()
plt.show()

# Normalizar dirección
vmin, vmax = 10, np.max(directionality)
directionality = np.clip(directionality, vmin, vmax)
directionality = np.log(directionality)
directionality -= directionality.min()
directionality /= directionality.max()
directionality[image == 0] = 0

# Visualizar imagen con orientación HSV superpuesta
plt.figure(figsize=(6, 6))
plt.title("Overlay with orientation \n Greyscale image with HSV orientations overlaid")

plt.imshow(image, cmap="Greys_r", vmin=0)
plt.imshow(
    orientations["theta"],
    cmap="hsv",
    alpha=directionality * 0.5,
    vmin=-90,
    vmax=90,
)
plt.colorbar(shrink=0.7)
plt.show()

# Visualizar el tensor de estructura
plt.imshow(structureTensor.sum(axis=0), cmap="viridis")
plt.title("Tensor de estructura")
plt.colorbar()
plt.show()

# Convertir imagen a HSV
imDisplayHSV = np.zeros((image.shape[0], image.shape[1], 3), dtype="f4")
imDisplayHSV[:, :, 0] = (orientations["theta"] + 90) / 180  # Hue
imDisplayHSV[:, :, 1] = directionality  # Saturation
imDisplayHSV[:, :, 2] = image / image.max()  # Value

fig, ax = plt.subplots()
fig.suptitle("Image-orientation composition")
ax.set_title("Hue = Orientation\nSaturation = log(Directionality)\nV = image greylevels")
ax.imshow(matplotlib.colors.hsv_to_rgb(imDisplayHSV))

cmap = matplotlib.cm.hsv
norm = matplotlib.colors.Normalize(vmin=-90, vmax=90)
fig.colorbar(
    matplotlib.cm.ScalarMappable(norm=norm, cmap=cmap),
    ax=ax,
    orientation="vertical",
    label="Degrees from horizontal",
    shrink=0.7,
)

plt.show()

# Cálculo de orientación en bloques
boxSizePixels = 7
structureTensorBoxes = orientationpy.computeStructureTensorBoxes(
    [Gy, Gx], [boxSizePixels, boxSizePixels]
)
intensityBoxes = orientationpy.computeIntensity(structureTensorBoxes)
orientationsBoxes = orientationpy.computeOrientation(structureTensorBoxes, mode="fiber")
intensityBoxes /= intensityBoxes.max()

# Calcular centros de bloques
boxCentresY = np.arange(orientationsBoxes["theta"].shape[0]) * boxSizePixels + boxSizePixels // 2
boxCentresX = np.arange(orientationsBoxes["theta"].shape[1]) * boxSizePixels + boxSizePixels // 2

# Calcular componentes del vector
boxVectorsYX = orientationpy.anglesToVectors(orientationsBoxes)
boxVectorsYX[:, intensityBoxes < 0.05] = 0.0

plt.title("Local orientation vector in boxes")
plt.imshow(image, cmap="Greys_r", vmin=0)

plt.quiver(
    boxCentresX,
    boxCentresY,
    boxVectorsYX[1],
    boxVectorsYX[0],
    angles="xy",
    scale_units="xy",
    color="r",
    headwidth=0,
    headlength=0,
    headaxislength=1,
)
plt.show()


In [ ]:
process_images_with_rotated_filter(folder_path_treated)

In [ ]:
results_all = batch_process_shg_images(folder_path_treated, region_label="Anterior Stroma")
frames, dois, orientations, uncertainties = process_folder(folder_path_untreated)
plot_results(frames, dois, orientations, uncertainties)

In [ ]:
img_3d = stack_images_to_3d(folder_path_treated)
execute_tests(img_3d)

In [ ]:
# Medidas
h, w, z = img_3d.shape

# Coordenadas (Y, X) de las esquinas superiores izquierdas de las ventanas
window_coords = [
    (1400, 1000),
    (200, w - window_size - 400),
    (550, 1100),
    (h - window_size - 200, 1000)
]

# Visualizar ventanas sobre una imagen representativa (ej: la del medio)
middle_slice = img_3d[:, :, 0]

fig, ax = plt.subplots()
ax.imshow(middle_slice, cmap="gray")
ax.set_title("Ubicación de las ventanas en la imagen 3D (XY view)")

for idx, (y, x) in enumerate(window_coords):
    rect = patches.Rectangle(
        (x, y), window_size, window_size,
        linewidth=2, edgecolor='red', facecolor='none'
    )
    ax.add_patch(rect)
    ax.text(x, y - 5, f"Ventana {idx+1}", color='red', fontsize=8, weight='bold')

plt.tight_layout()
plt.show()

# Extraer las ventanas 3D y almacenarlas en variables
ventana1 = img_3d[window_coords[0][0]:window_coords[0][0]+window_size, window_coords[0][1]:window_coords[0][1]+window_size, :]
ventana2 = img_3d[window_coords[1][0]:window_coords[1][0]+window_size, window_coords[1][1]:window_coords[1][1]+window_size, :]
ventana3 = img_3d[window_coords[2][0]:window_coords[2][0]+window_size, window_coords[2][1]:window_coords[2][1]+window_size, :]
ventana4 = img_3d[window_coords[3][0]:window_coords[3][0]+window_size, window_coords[3][1]:window_coords[3][1]+window_size]

# Mostrar información sobre las ventanas extraídas
print(f"Ventana 1: {ventana1.shape}")
print(f"Ventana 2: {ventana2.shape}")
print(f"Ventana 3: {ventana3.shape}")
print(f"Ventana 4: {ventana4.shape}")



boxSizePixels = 5
structureTensorBoxes = computeGradientStructureTensorBoxes(
    ventana1,
    [boxSizePixels, boxSizePixels, boxSizePixels],
)

# 1. Calcular intensidad promedio por caja (traza del tensor de estructura)
intensity_boxes = computeIntensity(structureTensorBoxes)

# 2. Definir umbrales (ajustar según necesidades)
dark_threshold = 0.04 * np.max(intensity_boxes)  # 5% de la intensidad máxima
anisotropy_threshold = 0.2  # umbral de coherencia para anisotropía

# 3. Clasificar regiones
coherence_map = computeCoherenceMap(structureTensorBoxes)

# Máscaras booleanas para clasificación
mask_dark = intensity_boxes < dark_threshold
mask_isotropic = (~mask_dark) & (coherence_map < anisotropy_threshold)
mask_anisotropic = (~mask_dark) & (coherence_map >= anisotropy_threshold)

# 5. Generar estadísticas
total_boxes = np.prod(intensity_boxes.shape)
stats = {
    "dark": np.sum(mask_dark),
    "isotropic": np.sum(mask_isotropic),
    "anisotropic": np.sum(mask_anisotropic),
    "total": total_boxes
}

print("\nESTADÍSTICAS DE REGIONES:")
print(f"Oscuras: {stats['dark']} ({stats['dark']/stats['total']*100:.1f}%)")
print(f"Isotrópicas: {stats['isotropic']} ({stats['isotropic']/stats['total']*100:.1f}%)")
print(f"Anisotrópicas: {stats['anisotropic']} ({stats['anisotropic']/stats['total']*100:.1f}%)")

# 4. Visualización solo de regiones anisotrópicas
orientationsBoxes = computeOrientation(
    structureTensorBoxes,
    mode="fiber",
)

boxVectorsZYX = anglesToVectors(orientationsBoxes)

# Centros de caja (solo para regiones anisotrópicas)
boxCentresX, boxCentresY, boxCentresZ = np.mgrid[
    boxSizePixels//2 : mask_anisotropic.shape[0]*boxSizePixels : boxSizePixels,
    boxSizePixels//2 : mask_anisotropic.shape[1]*boxSizePixels : boxSizePixels,
    boxSizePixels//2 : mask_anisotropic.shape[2]*boxSizePixels : boxSizePixels,
]

# Filtrar solo regiones anisotrópicas
X = boxCentresX[mask_anisotropic]
Y = boxCentresY[mask_anisotropic]
Z = boxCentresZ[mask_anisotropic]
U = boxVectorsZYX[2][mask_anisotropic]
V = boxVectorsZYX[1][mask_anisotropic]
W = boxVectorsZYX[0][mask_anisotropic]

# Asignar color por Z (fotograma)
norm = plt.Normalize(Z.min(), Z.max())
colors = cm.viridis(norm(Z))

# Reducción de flechas si son muchas
max_arrows = 800
if len(X) > max_arrows:
    idx = np.random.choice(len(X), max_arrows, replace=False)
    X, Y, Z = X[idx], Y[idx], Z[idx]
    U, V, W = U[idx], V[idx], W[idx]
    colors = colors[idx]

# Visualización 3D
fig = plt.figure(figsize=(12, 10))
ax = fig.add_subplot(111, projection="3d")

ax.quiver(
    X, Y, Z, U, V, W,
    length=boxSizePixels*0.8,
    normalize=True,
    linewidth=1.2,
    color=colors,
    alpha=0.85,
    arrow_length_ratio=0.35
)

ax.set_title("Orientación 3D de Fibras (Regiones Anisotrópicas)", fontsize=15)
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.view_init(elev=30, azim=45)
ax.invert_zaxis()

# Barra de color para Z
mappable = cm.ScalarMappable(cmap='viridis', norm=norm)
mappable.set_array(Z)
cbar = fig.colorbar(mappable, ax=ax, shrink=0.7, pad=0.1)
cbar.set_label('Fotograma (Z)', fontsize=12)

plt.tight_layout()
plt.show()

# Visualización de clasificación de regiones (opcional)
fig2, ax2 = plt.subplots(1, 1, figsize=(8, 6))
categories = ['Oscuras', 'Isotrópicas', 'Anisotrópicas']
counts = [stats['dark'], stats['isotropic'], stats['anisotropic']]

ax2.bar(categories, counts, color=['gray', 'blue', 'red'])
ax2.set_title("Distribución de Regiones")
ax2.set_ylabel("Número de Cajas")
plt.show()

# Calcular métricas cuantitativas
metrics = compute_orientation_metrics(structureTensorBoxes)

# Imprimir resultados
print("Coherencia media por capa:")
for z, coh in enumerate(metrics['mean_coherence_per_layer']):
    print(f"  Capa {z}: {coh:.4f}")

metrics = compute_orientation_metrics(structureTensorBoxes)
print("\nMÉTRICAS DE ORIENTACIÓN:")
print(f"Coherencia media global: {metrics['global_mean_coherence']:.4f}")

In [ ]:
execute_tests(ventana1)

In [ ]:
partitions=partition_3d_stack(img_3d, num_partitions=4)
for i, subvol in enumerate(partitions):
    print("Computing structure tensor...", end="")
    structureTensor = computeGradientStructureTensor(subvol, sigma=1, mode="gaussian")
    print("done.")
    intensity = computeIntensity(structureTensor)
    directionality = computeStructureDirectionality(structureTensor)
    structureType = computeStructureType(structureTensor)
    structureTensor += 1e-6  # Pequeña cantidad para evitar divisiones por cero
    print("Computing theta and phi...", end="")
    orientations = computeOrientation(structureTensor, mode="fibre")
    print("done.")
    boxSizePixels = 5
    structureTensorBoxes = computeGradientStructureTensorBoxes(
        subvol,
        [boxSizePixels, boxSizePixels, boxSizePixels],
        )

    # --- Máscara para regiones NO oscuras ---
    mask_not_dark = intensity_boxes >= dark_threshold
    # --- Filtrar solo regiones NO oscuras ---
    X = boxCentresX[mask_not_dark]
    Y = boxCentresY[mask_not_dark]
    Z = boxCentresZ[mask_not_dark]
    U = boxVectorsZYX[2][mask_not_dark]
    V = boxVectorsZYX[1][mask_not_dark]
    W = boxVectorsZYX[0][mask_not_dark]


    orientationVectors_not_dark = np.stack([W, V, U], axis=1)  # ZYX


    orientationVectors = boxVectorsZYX.reshape(3, -1).T

    print(f"Resultados partición {i+1}")
    plotOrientations3d(orientationVectors, pointMarkerSize=1)


In [ ]:
if __name__ == "__main__":
    # Tamaño de la ventana cuadrada
    window_size = 100  # píxeles
    ventanas = main(folder_path_treated, window_size)
    #Vamos a hacer para las 4 ventanas todo el análisis que tenemos
    for i, ventana in enumerate(ventanas):
        print("\n"+"\n" + "\033[1m" + f" ----------  Resultados análisis ventana {i+1} ------------ " + "\033[0m\n"+"\n")        
        fibers_in_a_3D_volume(ventana, boxSizePixels=5)
        execute_tests(ventana)